**AAN_new에 대하여 N,A1,A2로 가능한 모든 경우에 대해 에러데이터 생성.**
- AAN_new는 시계열로 자르되, 각 파일의 앞 20%를 테스트로 한다.
- 80%는 트레이닝으로 한다.
- 기존과 다른점은, 에러를 넣을 구간을 순차적으로 고르지않고, random하게 선택한다.
- 대부분의 데이터가 비슷한 값을 가지므로, 중간에 랜덤하게 삽압된 에러는 trend를 통해 걸러낼 수 있을 것으로 생각한다.


# **A1: P1.B3004** col=3
# **A2: P1.B3005** col=4
# **N: P1.B4002** col=5


In [2]:
!pip install ipython-autotime
%load_ext autotime
import pickle
import numpy as np
import pandas as pd
import os 
import random
import math

origin = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
origin


,time,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,2019-11-01 20:00:00+09:00,0.0998,1.3901,404.178711,1040.812500,32.0,9.672400,149.162598,35.725800,12.025100,11.186200,100.0,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.1733,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,0
1,2019-11-01 20:00:01+09:00,0.0998,1.3840,404.178711,1040.812500,32.0,9.696700,150.155502,35.710499,12.156200,11.186200,100.0,96.375999,51.393002,52.773998,155.487000,799.163818,29.029900,149.162598,269.737213,1042.957397,12.136500,11.3235,403.680695,77.599503,79.003899,12.0,12.2238,1.3864,0.1779,35.498001,36.688202,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8256,-3.7916,-1.7599,-0.0161,1.2254,2120.0,12896.0,70.0,10.0,18.236401,0.0005,65.407997,59.570301,0.0,394.223785,-0.0027,325.882507,329.842987,0.0,10053.0,27617.0,0,0,0,0,0
2,2019-11-01 20:00:02+09:00,0.0998,1.3837,404.178711,1040.812500,32.0,9.704100,152.141693,35.709801,12.180500,11.186200,100.0,96.375999,51.424599,52.773998,159.110992,793.510376,29.029900,151.148804,269.927887,1042.000732,12.198100,11.3235,403.782898,77.600304,78.996300,12.0,12.2238,1.3872,0.1749,35.482800,36.688202,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8218,-4.4148,-2.1085,-0.3602,1.3557,2016.0,12976.0,70.0,10.0,18.268999,0.0007,65.357399,60.004299,0.0,391.167511,0.0008,325.900604,328.540894,0.0,10053.0,27616.0,0,0,0,0,0
3,2019-11-01 20:00:03+09:00,0.0998,1.3854,404.178711,1040.812500,32.0,9.715200,154.128006,35.714001,12.247300,11.186200,100.0,96.375999,51.406399,52.773998,156.059296,801.773193,30.174200,151.148804,271.263092,1042.319580,12.170300,11.3235,403.782898,77.603798,79.003899,12.0,12.2238,1.3879,0.1772,35.482800,36.688202,28.024599,1.0,0.0,1.0,0.0,792.0,0.0,11.7147,-4.9002,-2.1456,0.1541,1.8032,1912.0,13056.0,70.0,10.0,18.243601,0.0005,65.328400,60.474499,0.0,390.606903,-0.0001,325.159088,328.215393,0.0,10053.0,27618.0,0,0,0,0,0
4,2019-11-01 20:00:04+09:00,0.0998,1.3839,404.178711,1040.812500,32.0,9.748800,149.162598,35.710098,12.209800,11.186200,100.0,96.375999,51.306599,52.773998,155.868500,794.815186,29.411301,157.107101,271.072388,1044.551880,12.079600,11.3235,403.578491,77.590897,78.996300,12.0,12.2162,1.3872,0.1787,35.482800,36.688202,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6509,-4.5318,-1.7957,0.8014,1.9603,1816.0,13136.0,70.0,10.0,18.225500,0.0007,65.263298,60.854301,0.0,390.136688,0.0007,324.688995,326.931396,0.0,10053.0,27616.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181795,2019-10-30 03:11:55+09:00,0.0305,1.3269,392.330292,1115.077148,32.0,84.305901,2556.180908,35.741001,40.301601,40.284698,0.0,-1.852400,58.626598,59.030201,185.050903,860.916077,1616.897583,2563.885010,312.271088,1113.117310,17.037399,16.8930,391.054810,42.979401,43.557701,12.0,11.6287,1.2697,0.7166,35.223400,35.543800,28.020300,1.0,0.0,1.0,0.0,791.0,0.0,11.6964,-5.0314,-2.2791,0.0256,1.7013,7464.0,1712.0,70.0,10.0,27.112301,-0.0009,1.511900,9.403900,0.0,335.159912,0.0011,329.788788,331.072815,0.0,1002

# **normal의 갯수 파악**

In [9]:
num0=0 # 노말
num1=0 # 어택
for i in origin["attack"]:
  if(i == 0):
    num0 = num0+1
  elif(i == 1):
    num1 = num1+1

print('normal 수')
print(num0)
print('attack 수')
print(num1)

normal 수
152888
attack 수
28912
time: 42.6 ms


# **테스트: 181800개**
# **노말:152888**
# **어택:28912**

# **기존방법의 문제**
- 위에서부터 순차적으로 에러를 넣었기 때문에, trend를 이용할 때 의미가 없음.
- 에러가 모여있으므로 계속 ---의 트렌드를 가지게된다.

# **에러생성 방법**
1. 데이터를 200개씩 909섹터로 나눈다.
1. 각 섹터의 normal만을 고른다.
2. 각 섹터의 normal의 갯수의 5%갯수만큼을 에러로한다.
3. 각 섹터의 normal의 인덱스를 갯수만큼 random하게 선택한다.
4. 선택된 row에 대하여, (원래의 값 + 섹터avg * 0.1) 값이 normal경계를 넘어서도록한다. 
5. 에러 플래그를 넣는다.

# **테스트용**

In [34]:
x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)

indexNames = x_test[x_test['attack'] == 1 ].index
print('attack의 index 확인')
indexNames
# # Delete these row indexes from dataFrame
# a.drop(indexNames , inplace=True)


attack의 index 확인


Int64Index([110160, 110161, 110162, 110163, 110164, 110165, 110166, 110167,
            110168, 110169,
            ...
            143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550,
            143551, 143552],
           dtype='int64', length=28912)

time: 180 ms


In [0]:
picklename = '/drive/My Drive/data/AAN_new/A1/reset_Error_whole_x_test_5percent.pickle'
percent = 0.05
target = 2
max = 500 



x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909      

for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  # print('normal_index')
  # print(normal_index)
  # print('normal_index 길이')
  # print(len(normal_index))
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   
  # print('rand_index')
  # print(rand_index)
  # print('rand_index 길이')
  # print(len(rand_index))
  # print("")

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  # print(avg)
  # print(x_test.iloc[normal_index, target])


  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

# x_test.head(50)

# 에러갯수 확인
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러없는 값(attack+normal)')
print(num0)
print('노말에러')
print(num1)

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 없음')
print(num0)
print('에러 있음')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


# **[1] generate A1**
# col=3


# **싱글 5%**

In [0]:
picklename = '/drive/My Drive/data/AAN_new/A1/reset_Error_whole_x_test_5percent.pickle'
percent = 0.05
target = 2
max = 500 

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


10

In [51]:
picklename = '/drive/My Drive/data/AAN_new/A1/reset_Error_whole_x_test_10percent.pickle'
percent = 0.1
target = 2
max = 500 

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
166512
에러 O
15288


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
5,0.0998,1.3804,541.599731,1040.812500,32.0,9.762300,151.148804,35.701401,12.329600,11.186200,100.0,96.375999,51.305099,52.773998,158.920303,794.380188,29.029900,153.134705,269.165009,1044.233032,12.117700,11.3159,403.731812,77.574402,79.003899,12.0,12.22380,1.3872,0.1764,35.482800,36.673000,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.7141,-3.8737,-1.5734,0.6949,1.5980,1720.0,13184.0,70.0,10.0,18.261700,0.0005,65.191002,61.216000,0.0,388.798492,-0.0013,323.585785,326.913300,0.0,10053.0,27617.0,0,0,0,0,1
8,0.0998,1.3723,541.599731,1040.812500,32.0,9.820800,153.134705,35.681198,12.494100,11.506700,100.0,96.383698,51.688801,51.866100,157.394394,799.598572,29.029900,154.128006,264.587402,1034.665771,12.484600,11.3235,403.782898,77.522003,78.996300,12.0,12.22380,1.3864,0.1756,35.498001,36.673000,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-4.8966,-2.1376,0.1598,1.8016,1512.0,13328.0,70.0,10.0,18.236401,0.0005,64.822098,62.011700,0.0,386.465607,0.0014,320.999695,324.924103,0.0,10053.0,27617.0,0,0,0,0,1
16,0.0998,1.3589,541.599731,1040.812500,32.0,9.938600,151.148804,35.647598,13.007000,12.254300,100.0,96.375999,51.917400,51.835602,162.925705,815.254211,29.601999,151.148804,261.917114,1034.665771,12.647800,12.5061,404.243011,77.500000,78.996300,12.0,12.21620,1.3643,0.1779,35.482800,36.657700,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.8299,-3.7922,-1.7679,-0.0526,1.2261,1008.0,13472.0,70.0,10.0,18.015800,0.0002,63.845501,62.970200,0.0,378.237091,-0.0019,316.550903,320.547607,0.0,10053.0,27613.0,0,0,0,0,1
27,0.0998,1.3386,541.599731,1040.812500,32.0,10.037200,149.162598,35.596802,13.605600,12.254300,100.0,96.375999,52.219101,51.835602,163.307205,807.861389,29.411301,153.134705,264.968903,1034.984619,12.763000,12.7426,405.469788,77.321404,78.996300,12.0,12.23140,1.3628,0.1779,35.482800,36.657700,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8282,-4.3655,-2.0912,-0.3608,1.3324,584.0,13440.0,70.0,10.0,17.806000,-0.0002,62.868900,62.970200,0.0,377.622314,-0.0014,313.187195,317.328613,0.0,10052.0,27618.0,0,0,0,0,1
48,0.0998,1.3196,541.599731,1040.812500,32.0,10.189100,151.148804,35.549500,14.734600,14.466900,100.0,96.375999,51.852100,53.010601,165.214493,817.428528,29.792700,157.107101,270.500214,1041.681763,12.059500,12.7426,406.798889,76.864197,78.996300,12.0,12.22380,1.3612,0.1779,35.498001,36.611900,28.028099,1.0,0.0,1.0,0.0,791.0,0.0,11.7541,-4.8312,-2.1893,-0.0669,1.6658,200.0,13264.0,70.0,10.0,17.487700,-0.0002,61.950199,62.192600,0.0,371.401215,-0.0025,309.714996,311.324493,0.0,10053.0,27616.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181749,0.0305,1.3150,549.262512,1115.077148,32.0,81.640800,2471.586182,35.711300,38.533798,38.529999,0.0,-1.852400,58.361801,59.075901,174.942001,843.521179,1609.649536,2497.568848,312.271088,1109.928101,16.212299,15.5884,389.725800,41.411999,42.382801,12.0,11.62870,1.2903,0.7928,35.208099,35.574299,28.023800,1.0,0.0,1.0,0.0,792.0,0.0,11.6377,-4.6357,-1.9253,0.6249,1.8136,13384.0,-208.0,70.0,10.0,27.835600,0.0000,-0.007200,0.162800,0.0,352.973114,0.0022,352.629486,329.915314,0.0,10026.0,27592.0,0,0,0,0,1
181752,0.0305,1.3296,549.262512,1115.077148,32.0,81.842400,2557.842529,35.748852,38.162300,38.690201,0.0,-1.852400,58.211800,59.

time: 32.9 s


15

In [52]:
picklename = '/drive/My Drive/data/AAN_new/A1/reset_Error_whole_x_test_15percent.pickle'
percent = 0.15
target = 2
max = 500 

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
158867
에러 O
22933


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,0.0998,1.3901,541.599731,1040.812500,32.0,9.672400,149.162598,35.725800,12.025100,11.186200,100.0,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.1733,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,1
1,0.0998,1.3840,541.599731,1040.812500,32.0,9.696700,150.155502,35.710499,12.156200,11.186200,100.0,96.375999,51.393002,52.773998,155.487000,799.163818,29.029900,149.162598,269.737213,1042.957397,12.136500,11.3235,403.680695,77.599503,79.003899,12.0,12.2238,1.3864,0.1779,35.498001,36.688202,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8256,-3.7916,-1.7599,-0.0161,1.2254,2120.0,12896.0,70.0,10.0,18.236401,0.0005,65.407997,59.570301,0.0,394.223785,-0.0027,325.882507,329.842987,0.0,10053.0,27617.0,0,0,0,0,1
23,0.0998,1.3456,541.599731,1040.812500,32.0,10.017000,146.183105,35.614399,13.348200,12.254300,100.0,96.375999,52.151402,51.835602,166.931198,809.166016,28.648399,155.120895,265.922485,1034.346802,12.779300,12.7426,405.060913,77.401100,78.996300,12.0,12.2238,1.3628,0.1764,35.467499,36.657700,28.019501,1.0,0.0,1.0,0.0,791.0,0.0,11.7213,-4.8861,-2.1453,0.1226,1.7930,696.0,13472.0,70.0,10.0,17.896400,0.0000,63.187199,63.024502,0.0,375.108490,-0.0001,312.680786,317.798706,0.0,10052.0,27617.0,0,0,0,0,1
25,0.0998,1.3413,541.599731,1040.812500,32.0,10.034100,148.169296,35.603699,13.509800,12.254300,100.0,96.375999,52.238201,51.835602,161.781296,815.689087,29.029900,152.141693,266.303986,1033.390137,12.811700,12.7426,405.418701,77.359398,79.003899,12.0,12.2238,1.3635,0.1779,35.482800,36.657700,28.021200,1.0,0.0,1.0,0.0,791.0,0.0,11.7131,-3.8701,-1.5639,0.7177,1.6119,632.0,13456.0,70.0,10.0,17.842199,0.0000,63.035301,62.970200,0.0,375.705292,0.0009,314.290314,317.093414,0.0,10053.0,27618.0,0,0,0,0,1
38,0.0998,1.3233,541.599731,1040.812500,32.0,10.135700,151.148804,35.558701,14.334400,13.200400,100.0,96.375999,51.933998,53.010601,163.116501,817.863525,29.601999,147.176407,271.263092,1042.319580,12.303500,12.7426,406.134308,77.072502,79.003899,12.0,12.2238,1.3620,0.1764,35.482800,36.642399,28.026400,1.0,0.0,1.0,0.0,793.0,0.0,11.7392,-4.8500,-2.1654,0.0118,1.7303,328.0,13344.0,70.0,10.0,17.657700,0.0000,62.283001,62.554298,0.0,371.961914,0.0008,310.040497,313.060699,0.0,10053.0,27618.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181764,0.0305,1.3896,549.262512,1115.077148,32.0,82.723900,2540.017090,35.897800,38.817200,38.980099,0.0,-1.852400,58.347599,59.022499,180.282501,850.044189,1589.431763,2536.089600,313.224792,1114.711792,16.415400,15.5884,389.725800,42.266102,42.382801,12.0,11.6287,1.2918,0.7494,35.208099,35.574299,28.024599,1.0,0.0,1.0,0.0,791.0,0.0,11.6295,-4.8413,-2.0566,0.4647,1.8395,6728.0,-208.0,70.0,10.0,29.600700,0.0001,-0.007200,0.072400,0.0,362.214294,-0.0007,364.076996,349.392303,0.0,10026.0,27584.0,0,0,0,0,1
181765,0.0305,1.3953,549.262512,1115.077148,32.0,82.791702,2536.089600,35.911900,38.988899,38.980099,0.0,-1.852400,58.354900,59.030201,177.0

time: 48.6 s


30

In [53]:
picklename = '/drive/My Drive/data/AAN_new/A1/reset_Error_whole_x_test_30percent.pickle'
percent = 0.3
target = 2
max = 500 

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
135934
에러 O
45866


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
2,0.0998,1.3837,541.599731,1040.812500,32.0,9.704100,152.141693,35.709801,12.180500,11.186200,100.0,96.375999,51.424599,52.773998,159.110992,793.510376,29.029900,151.148804,269.927887,1042.000732,12.198100,11.3235,403.782898,77.600304,78.996300,12.0,12.2238,1.3872,0.1749,35.482800,36.688202,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8218,-4.4148,-2.1085,-0.3602,1.3557,2016.0,12976.0,70.0,10.0,18.268999,0.0007,65.357399,60.004299,0.0,391.167511,0.0008,325.900604,328.540894,0.0,10053.0,27616.0,0,0,0,0,1
5,0.0998,1.3804,541.599731,1040.812500,32.0,9.762300,151.148804,35.701401,12.329600,11.186200,100.0,96.375999,51.305099,52.773998,158.920303,794.380188,29.029900,153.134705,269.165009,1044.233032,12.117700,11.3159,403.731812,77.574402,79.003899,12.0,12.2238,1.3872,0.1764,35.482800,36.673000,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.7141,-3.8737,-1.5734,0.6949,1.5980,1720.0,13184.0,70.0,10.0,18.261700,0.0005,65.191002,61.216000,0.0,388.798492,-0.0013,323.585785,326.913300,0.0,10053.0,27617.0,0,0,0,0,1
7,0.0998,1.3746,541.599731,1040.812500,32.0,9.790400,157.107101,35.686901,12.435100,11.186200,100.0,96.375999,51.397598,51.988201,158.157303,799.163818,29.601999,154.128006,265.350311,1041.681763,12.195800,11.3235,403.731812,77.536301,78.996300,12.0,12.2238,1.3864,0.1772,35.467499,36.688202,28.020300,1.0,0.0,1.0,0.0,791.0,0.0,11.8201,-4.4535,-2.1344,-0.3707,1.3498,1552.0,13280.0,70.0,10.0,18.236401,0.0005,64.945000,61.812801,0.0,386.338898,0.0001,321.252899,325.484711,0.0,10053.0,27617.0,0,0,0,0,1
9,0.0998,1.3711,541.599731,1040.812500,32.0,9.839000,151.148804,35.678101,12.582700,12.223800,100.0,96.375999,51.766998,51.881401,156.631500,797.859314,29.029900,151.148804,265.159607,1033.390137,12.558300,11.3235,403.936310,77.508499,78.996300,12.0,12.2238,1.3864,0.1764,35.467499,36.688202,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.6508,-4.5111,-1.7845,0.7932,1.9398,1440.0,13360.0,70.0,10.0,18.211100,0.0005,64.699097,62.246799,0.0,385.742188,-0.0020,320.005096,324.580414,0.0,10053.0,27612.0,0,0,0,0,1
12,0.0998,1.3647,541.599731,1040.812500,32.0,9.848000,151.148804,35.662102,12.773500,12.254300,100.0,96.375999,51.990002,51.866100,165.023804,803.947510,29.029900,151.148804,264.587402,1030.200928,12.791100,12.4832,403.987396,77.439499,79.003899,12.0,12.2238,1.3834,0.1779,35.467499,36.688202,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.8210,-4.4473,-2.1370,-0.3822,1.3351,1232.0,13424.0,70.0,10.0,18.095301,0.0002,64.308403,62.662800,0.0,382.613586,0.0012,317.889191,323.187897,0.0,10053.0,27616.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181784,0.0305,1.3129,549.262512,1115.077148,32.0,83.693703,2545.757568,35.705898,39.925499,38.980099,0.0,-1.852400,58.502102,59.030201,187.149002,865.699829,1586.570679,2541.829834,313.224792,1114.711792,16.866100,16.8930,390.288086,42.642799,42.375198,12.0,11.6287,1.2666,0.7478,35.208099,35.559101,28.021200,1.0,0.0,1.0,0.0,791.0,0.0,11.7401,-4.9903,-2.3544,-0.2536,1.5395,2904.0,4576.0,70.0,10.0,28.768801,0.0012,13.729700,23.437500,0.0,345.522308,0.0030,329.861115,328.396210,0.0,10026.0,27593.0,0,0,0,0,1
181788,0.0305,1.3252,549.262512,1115.077148,32.0,83.934700,2575.516846,35.736801,39.887299,40.223701,0.0,-1.852400,58.534100,59.144600,189.

time: 1min 33s


# **A2**

5

In [3]:
picklename = '/drive/My Drive/data/AAN_new/A2/reset_Error_whole_x_test_5percent.pickle'
percent = 0.05
target = 3
max = 1100

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
174156
에러 O
7644


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
38,0.0998,1.3233,404.178711,1207.342529,32.0,10.135700,151.148804,35.558701,14.334400,13.200400,100.0,96.375999,51.933998,53.010601,163.116501,817.863525,29.601999,147.176407,271.263092,1042.319580,12.303500,12.7426,406.134308,77.072502,79.003899,12.0,12.2238,1.3620,0.1764,35.482800,36.642399,28.026400,1.0,0.0,1.0,0.0,793.0,0.0,11.7392,-4.8500,-2.1654,0.0118,1.7303,328.0,13344.0,70.0,10.0,17.657700,0.0000,62.283001,62.554298,0.0,371.961914,0.0008,310.040497,313.060699,0.0,10053.0,27618.0,0,0,0,0,1
40,0.0998,1.3216,404.178711,1207.342529,32.0,10.134500,149.162598,35.554501,14.348200,13.208000,100.0,96.375999,51.899601,53.010601,165.214493,810.905518,29.411301,153.134705,271.263092,1042.638550,12.264800,12.7426,406.185394,77.033897,79.003899,12.0,12.2238,1.3628,0.1772,35.482800,36.642399,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.6980,-3.9811,-1.5830,0.8086,1.7148,296.0,13328.0,70.0,10.0,17.617901,0.0000,62.210701,62.481899,0.0,372.721405,0.0003,310.402191,312.861694,0.0,10053.0,27618.0,0,0,0,0,1
65,0.0998,1.4480,404.178711,1207.342529,32.0,10.859400,159.093399,35.870399,16.060600,14.550800,0.0,79.629501,51.414700,53.010601,156.250000,797.424316,30.746500,155.120895,271.263092,1046.465454,11.333000,11.4380,407.105499,77.396698,78.286697,12.0,12.2238,1.3895,0.1787,35.482800,36.596699,28.027201,1.0,0.0,1.0,0.0,792.0,0.0,11.6609,-4.2710,-1.6636,0.8832,1.8868,80.0,14656.0,70.0,10.0,17.238100,-0.0002,71.021400,67.437103,0.0,440.917999,0.0007,370.931000,348.958313,0.0,10053.0,27617.0,0,0,0,0,1
69,0.0998,1.4809,404.178711,1207.342529,32.0,11.110100,162.072495,35.952801,16.337099,15.580700,0.0,72.374001,51.648899,51.957699,156.631500,794.815186,32.653801,162.072495,267.066986,1039.130493,11.528700,11.4380,407.309998,77.704002,78.286697,12.0,12.2238,1.3887,0.1810,35.498001,36.596699,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6836,-4.8543,-2.0506,0.3721,1.9036,64.0,15040.0,70.0,10.0,17.176600,0.0000,72.851601,69.245499,0.0,451.696411,-0.0028,378.815887,356.752686,0.0,10053.0,27613.0,0,0,0,0,1
78,0.0998,1.5459,404.178711,1207.342529,32.0,11.745900,180.941498,36.115299,16.688801,15.580700,0.0,55.719002,51.984402,52.072102,157.585098,795.684814,37.231400,193.851501,264.778107,1033.708984,11.762000,11.4380,407.207794,78.600601,79.598999,12.0,12.2238,1.3895,0.2054,35.513302,36.581402,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8045,-4.5792,-2.1658,-0.3185,1.4392,40.0,15840.0,70.0,10.0,17.017500,0.0000,76.193604,73.061302,0.0,465.766113,0.0011,389.503815,373.010712,0.0,10053.0,27619.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181744,0.0305,1.2688,392.330292,1226.584961,32.0,81.276703,2550.893555,35.595699,37.548401,38.468899,0.0,-1.860000,58.256302,59.144600,179.519699,843.955994,1577.224731,2538.355469,310.936005,1111.203857,16.059999,15.5884,389.521301,41.176399,42.375198,12.0,11.6287,1.2903,0.7730,35.223400,35.589600,28.023800,1.0,0.0,1.0,0.0,793.0,0.0,11.6360,-4.6821,-1.9518,0.5931,1.8257,13256.0,-192.0,70.0,10.0,26.493799,-0.0003,-0.007200,0.307400,0.0,343.532990,-0.0004,344.256287,318.865692,0.0,10026.0,27597.0,0,0,0,0,1
181763,0.0305,1.3859,392.330292,1226.584961,32.0,82.646301,2545.153320,35.888699,38.842701,38.980099,0.0,-1.852400,58.368050,59.030201,179.138199

time: 16.5 s


10

In [4]:
picklename = '/drive/My Drive/data/AAN_new/A2/reset_Error_whole_x_test_10percent.pickle'
percent = 0.1
target = 3
max = 1100

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
166512
에러 O
15288


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,0.0998,1.3901,404.178711,1207.342529,32.0,9.672400,149.162598,35.725800,12.025100,11.186200,100.0,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.17330,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,1
4,0.0998,1.3839,404.178711,1207.342529,32.0,9.748800,149.162598,35.710098,12.209800,11.186200,100.0,96.375999,51.306599,52.773998,155.868500,794.815186,29.411301,157.107101,271.072388,1044.551880,12.079600,11.3235,403.578491,77.590897,78.996300,12.0,12.2162,1.3872,0.17870,35.482800,36.688202,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6509,-4.5318,-1.7957,0.8014,1.9603,1816.0,13136.0,70.0,10.0,18.225500,0.0007,65.263298,60.854301,0.0,390.136688,0.0007,324.688995,326.931396,0.0,10053.0,27616.0,0,0,0,0,1
22,0.0998,1.3453,404.178711,1207.342529,32.0,10.016100,148.169296,35.613602,13.402800,12.254300,100.0,96.375999,52.125500,51.835602,162.353500,819.168091,29.792700,148.169296,265.159607,1034.346802,12.779100,12.7426,405.009796,77.404602,78.996300,12.0,12.2238,1.3628,0.17790,35.467499,36.657700,28.021999,1.0,0.0,1.0,0.0,790.0,0.0,11.8231,-4.4297,-2.1198,-0.3710,1.3449,736.0,13472.0,70.0,10.0,17.896400,0.0000,63.281300,63.024502,0.0,377.658386,-0.0009,312.698914,318.232788,0.0,10052.0,27616.0,0,0,0,0,1
25,0.0998,1.3413,404.178711,1207.342529,32.0,10.034100,148.169296,35.603699,13.509800,12.254300,100.0,96.375999,52.238201,51.835602,161.781296,815.689087,29.029900,152.141693,266.303986,1033.390137,12.811700,12.7426,405.418701,77.359398,79.003899,12.0,12.2238,1.3635,0.17790,35.482800,36.657700,28.021200,1.0,0.0,1.0,0.0,791.0,0.0,11.7131,-3.8701,-1.5639,0.7177,1.6119,632.0,13456.0,70.0,10.0,17.842199,0.0000,63.035301,62.970200,0.0,375.705292,0.0009,314.290314,317.093414,0.0,10053.0,27618.0,0,0,0,0,1
27,0.0998,1.3386,404.178711,1207.342529,32.0,10.037200,149.162598,35.596802,13.605600,12.254300,100.0,96.375999,52.219101,51.835602,163.307205,807.861389,29.411301,153.134705,264.968903,1034.984619,12.763000,12.7426,405.469788,77.321404,78.996300,12.0,12.2314,1.3628,0.17790,35.482800,36.657700,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8282,-4.3655,-2.0912,-0.3608,1.3324,584.0,13440.0,70.0,10.0,17.806000,-0.0002,62.868900,62.970200,0.0,377.622314,-0.0014,313.187195,317.328613,0.0,10052.0,27618.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181736,0.0305,1.1891,392.330292,1226.584961,32.0,80.904198,2548.174561,35.396599,37.303902,38.209499,0.0,-1.852400,58.150799,59.030201,179.901093,851.348877,1559.295532,2527.479004,311.317413,1111.522827,15.763500,15.5884,389.674713,41.082500,42.382801,12.0,11.6287,1.2910,0.78600,35.223400,35.574299,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.7977,-3.9376,-1.8791,-0.1176,1.1229,12992.0,-128.0,70.0,10.0,24.179001,-0.0002,-0.007200,0.542500,0.0,321.451813,0.0002,322.283691,299.298309,0.0,10026.0,27591.0,0,0,0,0,1
181739,0.0305,1.2189,392.330292,1226.584961,32.0,81.040604,2517.962158,35.471001,38.283100,37.095600,0.0,-1.860000,58.250301,59.03

time: 31.8 s


15

In [5]:
picklename = '/drive/My Drive/data/AAN_new/A2/reset_Error_whole_x_test_15percent.pickle'
percent = 0.15
target = 3
max = 1100

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
158867
에러 O
22933


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
11,0.0998,1.3647,404.178711,1207.342529,32.0,9.860600,154.128006,35.662102,12.729400,12.254300,100.0,96.375999,51.862701,51.881401,160.064697,795.684814,29.029900,149.162598,262.679993,1032.433472,12.637200,11.9644,403.782898,77.458397,79.003899,12.0,12.2238,1.3879,0.1772,35.467499,36.673000,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.8274,-3.7878,-1.7647,-0.0491,1.2151,1296.0,13408.0,70.0,10.0,18.149599,0.0002,64.431396,62.554298,0.0,383.825287,-0.0014,318.196594,323.784698,0.0,10053.0,27617.0,0,0,0,0,1
17,0.0998,1.3564,404.178711,1207.342529,32.0,9.950200,151.148804,35.641499,13.007800,12.254300,100.0,96.375999,52.183102,51.835602,164.070099,810.470520,29.029900,154.128006,263.633698,1028.925415,12.925400,12.5214,404.396393,77.488098,79.003899,12.0,12.2238,1.3628,0.1749,35.467499,36.673000,28.019501,1.0,0.0,1.0,0.0,792.0,0.0,11.8188,-4.4556,-2.1329,-0.3630,1.3600,952.0,13472.0,70.0,10.0,17.968800,0.0002,63.722500,63.024502,0.0,379.032806,0.0008,315.285004,319.968903,0.0,10053.0,27617.0,0,0,0,0,1
18,0.0998,1.3528,404.178711,1207.342529,32.0,9.942000,155.120895,35.632301,13.115400,12.254300,100.0,96.375999,52.110100,51.835602,163.879395,813.079773,28.457600,151.148804,264.396606,1031.795654,12.832900,12.7350,404.549713,77.473000,78.996300,12.0,12.2238,1.3628,0.1764,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7140,-4.8897,-2.1313,0.1738,1.8179,904.0,13472.0,70.0,10.0,17.932600,0.0003,63.621201,63.024502,0.0,379.701996,0.0018,315.936005,319.824188,0.0,10052.0,27621.0,0,0,0,0,1
25,0.0998,1.3413,404.178711,1207.342529,32.0,10.034100,148.169296,35.603699,13.509800,12.254300,100.0,96.375999,52.238201,51.835602,161.781296,815.689087,29.029900,152.141693,266.303986,1033.390137,12.811700,12.7426,405.418701,77.359398,79.003899,12.0,12.2238,1.3635,0.1779,35.482800,36.657700,28.021200,1.0,0.0,1.0,0.0,791.0,0.0,11.7131,-3.8701,-1.5639,0.7177,1.6119,632.0,13456.0,70.0,10.0,17.842199,0.0000,63.035301,62.970200,0.0,375.705292,0.0009,314.290314,317.093414,0.0,10053.0,27618.0,0,0,0,0,1
44,0.0998,1.3181,404.178711,1207.342529,32.0,10.156100,155.120895,35.545700,14.577100,13.200400,100.0,96.383698,51.784302,53.010601,161.972000,813.079773,29.220600,153.134705,270.881592,1044.233032,12.072500,12.7426,406.492188,76.941299,78.996300,12.0,12.2238,1.3620,0.1779,35.467499,36.611900,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.6451,-4.6687,-1.8778,0.6925,1.9664,240.0,13296.0,70.0,10.0,17.567301,0.0000,62.058701,62.319199,0.0,369.520386,0.0012,309.172394,311.089386,0.0,10053.0,27617.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181785,0.0305,1.3176,392.330292,1226.584961,32.0,83.772003,2541.829834,35.717800,40.041801,38.980099,0.0,-1.852400,58.491402,59.137001,187.149002,865.699829,1598.777710,2539.563965,313.796906,1115.668457,16.862101,16.8930,390.185913,42.717098,42.375198,12.0,11.6364,1.2651,0.7463,35.208099,35.574299,28.019501,1.0,0.0,1.0,0.0,791.0,0.0,11.6345,-4.9013,-2.1021,0.3981,1.8172,3456.0,4208.0,70.0,10.0,28.577101,0.0008,11.335300,21.665199,0.0,343.460602,0.0034,330.168488,329.282410,0.0,10026.0,27590.0,0,0,0,0,1
181786,0.0305,1.3254,392.330292,1226.584961,32.0,83.828903,2539.563965,35.737202,39.957100,39.918499,0.0,-1.852400,58.452702,59.144600,184.0

time: 45 s


30

In [6]:
picklename = '/drive/My Drive/data/AAN_new/A2/reset_Error_whole_x_test_30percent.pickle'
percent = 0.3
target = 3
max = 1100

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
135934
에러 O
45866


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
3,0.0998,1.3854,404.178711,1207.342529,32.0,9.715200,154.128006,35.714001,12.247300,11.186200,100.0,96.375999,51.406399,52.773998,156.059296,801.773193,30.174200,151.148804,271.263092,1042.319580,12.170300,11.323500,403.782898,77.603798,79.003899,12.0,12.2238,1.3879,0.1772,35.482800,36.688202,28.024599,1.0,0.0,1.0,0.0,792.0,0.0,11.7147,-4.9002,-2.1456,0.1541,1.8032,1912.0,13056.0,70.0,10.0,18.243601,0.0005,65.328400,60.474499,0.0,390.606903,-0.0001,325.159088,328.215393,0.0,10053.0,27618.0,0,0,0,0,1
5,0.0998,1.3804,404.178711,1207.342529,32.0,9.762300,151.148804,35.701401,12.329600,11.186200,100.0,96.375999,51.305099,52.773998,158.920303,794.380188,29.029900,153.134705,269.165009,1044.233032,12.117700,11.315900,403.731812,77.574402,79.003899,12.0,12.2238,1.3872,0.1764,35.482800,36.673000,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.7141,-3.8737,-1.5734,0.6949,1.5980,1720.0,13184.0,70.0,10.0,18.261700,0.0005,65.191002,61.216000,0.0,388.798492,-0.0013,323.585785,326.913300,0.0,10053.0,27617.0,0,0,0,0,1
6,0.0998,1.3785,404.178711,1207.342529,32.0,9.779400,151.148804,35.696800,12.416400,11.186200,100.0,96.375999,51.425301,52.751202,157.966599,801.338196,29.601999,151.148804,269.546509,1041.043823,12.216300,11.323500,403.731812,77.561401,78.996300,12.0,12.2238,1.3872,0.1794,35.467499,36.688202,28.027201,1.0,0.0,1.0,0.0,793.0,0.0,11.8285,-3.7991,-1.7702,-0.0452,1.2192,1640.0,13248.0,70.0,10.0,18.229200,0.0007,65.039101,61.505402,0.0,384.928406,0.0014,320.384888,325.575104,0.0,10053.0,27614.0,0,0,0,0,1
12,0.0998,1.3647,404.178711,1207.342529,32.0,9.848000,151.148804,35.662102,12.773500,12.254300,100.0,96.375999,51.990002,51.866100,165.023804,803.947510,29.029900,151.148804,264.587402,1030.200928,12.791100,12.483200,403.987396,77.439499,79.003899,12.0,12.2238,1.3834,0.1779,35.467499,36.688202,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.8210,-4.4473,-2.1370,-0.3822,1.3351,1232.0,13424.0,70.0,10.0,18.095301,0.0002,64.308403,62.662800,0.0,382.613586,0.0012,317.889191,323.187897,0.0,10053.0,27616.0,0,0,0,0,1
13,0.0998,1.3642,404.178711,1207.342529,32.0,9.895300,151.148804,35.660900,12.833900,12.254300,100.0,96.375999,51.898499,51.835602,163.879395,815.254211,29.029900,151.148804,264.587402,1033.390137,12.674800,12.506100,404.038513,77.449501,79.003899,12.0,12.2238,1.3734,0.1764,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7151,-4.8982,-2.1363,0.1604,1.8099,1168.0,13440.0,70.0,10.0,18.117001,0.0002,64.214401,62.789398,0.0,381.944397,0.0005,318.612488,322.645386,0.0,10053.0,27618.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181775,0.0305,1.3383,392.330292,1226.584961,32.0,83.313499,2554.972168,35.769699,39.293098,38.983898,0.0,-1.852400,58.372200,59.030201,177.421600,843.521179,1601.447998,2548.778809,311.126709,1111.203857,16.823601,15.588400,389.776886,42.461601,42.375198,12.0,11.6287,1.2933,0.7471,35.208099,35.574299,28.017700,1.0,0.0,1.0,0.0,792.0,0.0,11.6377,-4.5665,-1.8982,0.6466,1.7861,3912.0,7152.0,70.0,10.0,29.343901,0.0012,52.763302,31.684000,0.0,360.188812,0.0030,307.997009,327.347412,0.0,10026.0,27596.0,0,0,0,0,1
181779,0.0305,1.2974,392.330292,1226.584961,32.0,83.415199,2542.736328,35.667400,39.480099,38.980099,0.0,-1.852400,58.642101,59.

time: 1min 29s


# **N**

5

In [7]:
picklename = '/drive/My Drive/data/AAN_new/N/reset_Error_whole_x_test_5percent.pickle'
percent = 0.05
target = 4
max = 32 

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
174156
에러 O
7644


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
32,0.0998,1.3312,404.178711,1040.812500,35.200001,10.092400,158.100403,35.578499,13.810700,13.253800,100.000000,96.437103,52.010399,52.880901,165.595993,811.775330,29.029900,158.100403,271.453888,1042.000732,12.458000,12.7426,406.032104,77.228104,78.996300,12.0,12.2238,1.3620,0.1779,35.467499,36.657700,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.8339,-4.3015,-2.0663,-0.3689,1.3018,448.0,13408.0,70.0,10.0,17.719200,0.0000,62.565102,62.789398,0.0,373.498993,0.0015,311.758514,315.031799,0.0,10053.0,27614.0,0,0,0,0,1
38,0.0998,1.3233,404.178711,1040.812500,35.200001,10.135700,151.148804,35.558701,14.334400,13.200400,100.000000,96.375999,51.933998,53.010601,163.116501,817.863525,29.601999,147.176407,271.263092,1042.319580,12.303500,12.7426,406.134308,77.072502,79.003899,12.0,12.2238,1.3620,0.1764,35.482800,36.642399,28.026400,1.0,0.0,1.0,0.0,793.0,0.0,11.7392,-4.8500,-2.1654,0.0118,1.7303,328.0,13344.0,70.0,10.0,17.657700,0.0000,62.283001,62.554298,0.0,371.961914,0.0008,310.040497,313.060699,0.0,10053.0,27618.0,0,0,0,0,1
56,0.0998,1.3682,404.178711,1040.812500,35.200001,10.415600,156.114197,35.670898,15.338800,14.276100,52.180099,96.002197,51.595299,53.010601,166.168198,819.603088,29.601999,154.128006,271.644592,1045.827515,11.619400,12.7426,407.514496,77.005798,78.286697,12.0,12.2238,1.3612,0.1764,35.482800,36.611900,28.026400,1.0,0.0,1.0,0.0,792.0,0.0,11.7710,-3.7147,-1.6243,0.3295,1.3603,136.0,13744.0,70.0,10.0,17.372000,-0.0004,65.574402,63.458500,0.0,413.628510,0.0012,348.180695,328.938812,0.0,10051.0,27618.0,0,0,0,0,1
84,0.0998,1.5758,404.178711,1040.812500,35.200001,12.195900,248.471893,36.189999,15.374100,15.580700,0.000000,44.442699,51.988449,52.072102,157.394394,796.119690,59.547401,281.243713,264.968903,1034.027954,11.779200,11.4380,407.207794,79.256302,79.583702,12.0,12.2238,1.3902,0.2969,35.513302,36.566200,28.025499,1.0,0.0,1.0,0.0,793.0,0.0,11.7000,-4.8780,-2.1027,0.2483,1.8424,32.0,16240.0,70.0,10.0,16.862000,0.0000,77.741600,75.068703,0.0,472.981812,0.0001,394.024902,381.437988,0.0,10053.0,27619.0,0,0,0,0,1
100,0.0998,1.6399,404.178711,1040.812500,35.200001,13.613400,391.477112,36.350300,13.610800,14.970400,100.000000,37.858601,52.301102,52.072102,154.819443,792.640625,93.307503,438.152588,267.448395,1038.173706,11.655000,11.4380,407.514496,81.415001,82.879601,12.0,12.2314,1.3879,0.4266,35.498001,36.535599,28.020300,1.0,0.0,1.0,0.0,791.0,0.0,11.6516,-4.4455,-1.7477,0.8238,1.9240,16.0,16816.0,70.0,10.0,16.438801,0.0010,79.340302,78.956902,0.0,459.725800,0.0049,378.906311,386.989990,0.0,10053.0,27619.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181669,0.0305,1.3400,392.330292,1115.077148,35.200001,80.221603,2510.560059,35.773800,35.164001,34.768700,0.000000,-1.852400,57.743599,59.022499,177.993698,845.695496,1386.108276,2400.774658,313.224792,1112.479492,14.035300,14.1617,392.588287,46.625599,47.708099,12.0,11.6287,1.3536,0.9691,35.238602,35.589600,28.019501,1.0,0.0,1.0,0.0,792.0,0.0,11.7218,-3.9684,-1.7392,0.4005,1.3814,1680.0,13440.0,70.0,10.0,21.285400,0.0003,66.702904,62.084099,0.0,404.730896,-0.0021,337.619385,335.847107,0.0,10051.0,27595.0,0,0,0,0,1
181702,0.0305,1.1685,392.330292,1115.077148,35.200001,80.944000,2492.734863,35.34

time: 16.9 s


10

In [8]:
picklename = '/drive/My Drive/data/AAN_new/N/reset_Error_whole_x_test_10percent.pickle'
percent = 0.1
target = 4
max = 32 

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
166512
에러 O
15288


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
1,0.0998,1.3840,404.178711,1040.812500,35.200001,9.696700,150.155502,35.710499,12.156200,11.186200,100.0,96.375999,51.393002,52.773998,155.487000,799.163818,29.029900,149.162598,269.737213,1042.957397,12.136500,11.3235,403.680695,77.599503,79.003899,12.0,12.2238,1.3864,0.1779,35.498001,36.688202,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8256,-3.7916,-1.7599,-0.0161,1.2254,2120.0,12896.0,70.0,10.0,18.236401,0.0005,65.407997,59.570301,0.0,394.223785,-0.0027,325.882507,329.842987,0.0,10053.0,27617.0,0,0,0,0,1
15,0.0998,1.3621,404.178711,1040.812500,35.200001,9.928200,151.148804,35.655602,12.887500,12.254300,100.0,96.391296,51.783699,51.835602,165.023804,811.775330,29.029900,155.120895,265.350311,1037.536011,12.539500,12.5061,404.345215,77.503799,79.003899,12.0,12.2314,1.3651,0.1764,35.467499,36.673000,28.022100,1.0,0.0,1.0,0.0,791.0,0.0,11.7151,-3.8580,-1.5607,0.6961,1.6043,1056.0,13472.0,70.0,10.0,18.037500,0.0002,63.975700,62.897900,0.0,379.195587,0.0013,315.086090,321.849701,0.0,10053.0,27614.0,0,0,0,0,1
26,0.0998,1.3380,404.178711,1040.812500,35.200001,10.038900,155.120895,35.595299,13.577300,12.254300,100.0,96.375999,52.147301,51.835602,161.781296,807.861389,29.411301,151.148804,265.540985,1036.260376,12.704500,12.7426,405.469788,77.327003,79.003899,12.0,12.2238,1.3628,0.1787,35.482800,36.657700,28.025499,1.0,0.0,1.0,0.0,790.0,0.0,11.8185,-3.7596,-1.7358,0.0107,1.2314,600.0,13456.0,70.0,10.0,17.845800,-0.0002,62.963001,62.970200,0.0,376.374512,-0.0011,313.621185,317.509399,0.0,10053.0,27616.0,0,0,0,0,1
41,0.0998,1.3207,404.178711,1040.812500,35.200001,10.155300,147.176407,35.552200,14.402200,13.200400,100.0,96.375999,51.812000,53.010601,162.544296,815.689087,29.792700,153.134705,271.072388,1044.551880,12.172900,12.7426,406.441010,77.008598,78.996300,12.0,12.2238,1.3605,0.1764,35.467499,36.627201,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7974,-3.7218,-1.6854,0.1269,1.2675,280.0,13328.0,70.0,10.0,17.585400,0.0000,62.181702,62.427601,0.0,372.920288,-0.0020,308.684113,312.246796,0.0,10053.0,27618.0,0,0,0,0,1
53,0.0998,1.3393,404.178711,1040.812500,35.200001,10.294600,159.093399,35.598801,15.173600,14.276100,100.0,96.375999,51.817402,53.010601,164.260803,813.514587,29.983500,150.155502,270.500214,1041.362915,11.921200,12.7426,407.105499,76.875999,78.332497,12.0,12.2238,1.3620,0.1756,35.467499,36.596699,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7676,-4.7822,-2.1974,-0.1594,1.6047,152.0,13488.0,70.0,10.0,17.419001,-0.0007,63.715302,62.481899,0.0,400.463013,-0.0013,336.950287,322.048615,0.0,10053.0,27612.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181744,0.0305,1.2688,392.330292,1115.077148,35.200001,81.276703,2550.893555,35.595699,37.548401,38.468899,0.0,-1.860000,58.256302,59.144600,179.519699,843.955994,1577.224731,2538.355469,310.936005,1111.203857,16.059999,15.5884,389.521301,41.176399,42.375198,12.0,11.6287,1.2903,0.7730,35.223400,35.589600,28.023800,1.0,0.0,1.0,0.0,793.0,0.0,11.6360,-4.6821,-1.9518,0.5931,1.8257,13256.0,-192.0,70.0,10.0,26.493799,-0.0003,-0.007200,0.307400,0.0,343.532990,-0.0004,344.256287,318.865692,0.0,10026.0,27597.0,0,0,0,0,1
181762,0.0305,1.3788,392.330292,1115.077148,35.200001,82.561203,2547.872314,35.870701,38.709801,38.980

time: 32.9 s


15

In [9]:
picklename = '/drive/My Drive/data/AAN_new/N/reset_Error_whole_x_test_15percent.pickle'
percent = 0.15
target = 4
max = 32 

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
158867
에러 O
22933


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
2,0.0998,1.3837,404.178711,1040.812500,35.200001,9.704100,152.141693,35.709801,12.180500,11.186200,100.0,96.375999,51.424599,52.773998,159.110992,793.510376,29.029900,151.148804,269.927887,1042.000732,12.198100,11.323500,403.782898,77.600304,78.996300,12.0,12.2238,1.3872,0.1749,35.482800,36.688202,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8218,-4.4148,-2.1085,-0.3602,1.3557,2016.0,12976.0,70.0,10.0,18.268999,0.0007,65.357399,60.004299,0.0,391.167511,0.0008,325.900604,328.540894,0.0,10053.0,27616.0,0,0,0,0,1
3,0.0998,1.3854,404.178711,1040.812500,35.200001,9.715200,154.128006,35.714001,12.247300,11.186200,100.0,96.375999,51.406399,52.773998,156.059296,801.773193,30.174200,151.148804,271.263092,1042.319580,12.170300,11.323500,403.782898,77.603798,79.003899,12.0,12.2238,1.3879,0.1772,35.482800,36.688202,28.024599,1.0,0.0,1.0,0.0,792.0,0.0,11.7147,-4.9002,-2.1456,0.1541,1.8032,1912.0,13056.0,70.0,10.0,18.243601,0.0005,65.328400,60.474499,0.0,390.606903,-0.0001,325.159088,328.215393,0.0,10053.0,27618.0,0,0,0,0,1
6,0.0998,1.3785,404.178711,1040.812500,35.200001,9.779400,151.148804,35.696800,12.416400,11.186200,100.0,96.375999,51.425301,52.751202,157.966599,801.338196,29.601999,151.148804,269.546509,1041.043823,12.216300,11.323500,403.731812,77.561401,78.996300,12.0,12.2238,1.3872,0.1794,35.467499,36.688202,28.027201,1.0,0.0,1.0,0.0,793.0,0.0,11.8285,-3.7991,-1.7702,-0.0452,1.2192,1640.0,13248.0,70.0,10.0,18.229200,0.0007,65.039101,61.505402,0.0,384.928406,0.0014,320.384888,325.575104,0.0,10053.0,27614.0,0,0,0,0,1
8,0.0998,1.3723,404.178711,1040.812500,35.200001,9.820800,153.134705,35.681198,12.494100,11.506700,100.0,96.383698,51.688801,51.866100,157.394394,799.598572,29.029900,154.128006,264.587402,1034.665771,12.484600,11.323500,403.782898,77.522003,78.996300,12.0,12.2238,1.3864,0.1756,35.498001,36.673000,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-4.8966,-2.1376,0.1598,1.8016,1512.0,13328.0,70.0,10.0,18.236401,0.0005,64.822098,62.011700,0.0,386.465607,0.0014,320.999695,324.924103,0.0,10053.0,27617.0,0,0,0,0,1
10,0.0998,1.3688,404.178711,1040.812500,35.200001,9.822000,154.128006,35.672401,12.653100,12.254300,100.0,96.375999,51.756500,51.843300,156.440704,796.119690,28.648399,151.148804,264.015198,1034.346802,12.529000,11.323500,403.936310,77.488800,79.003899,12.0,12.2238,1.3864,0.1810,35.498001,36.688202,28.018600,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-3.8636,-1.5659,0.7002,1.5998,1368.0,13392.0,70.0,10.0,18.196600,0.0002,64.597801,62.427601,0.0,384.494385,-0.0001,317.852997,324.634613,0.0,10053.0,27617.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181779,0.0305,1.2974,392.330292,1115.077148,35.200001,83.415199,2542.736328,35.667400,39.480099,38.980099,0.0,-1.852400,58.642101,59.030201,182.571396,855.262817,1586.379883,2549.231934,312.843292,1114.073975,16.859100,16.877701,389.879211,42.306999,42.375198,12.0,11.6287,1.2827,0.7486,35.208099,35.559101,28.023800,1.0,0.0,1.0,0.0,791.0,0.0,11.7030,-5.0601,-2.3181,-0.0484,1.6570,3192.0,6864.0,70.0,10.0,29.097900,0.0048,37.478298,34.884998,0.0,353.768799,-0.0020,312.463806,324.435791,0.0,10026.0,27584.0,0,0,0,0,1
181783,0.0305,1.3083,392.330292,1115.077148,35.200001,83.647598,2546.210693,35.694500,39.800400,3

time: 45.9 s


30

In [10]:
picklename = '/drive/My Drive/data/AAN_new/N/reset_Error_whole_x_test_30percent.pickle'
percent = 0.3
target = 4
max = 32 

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32


avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
135934
에러 O
45866


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
4,0.0998,1.3839,404.178711,1040.812500,35.200001,9.748800,149.162598,35.710098,12.209800,11.186200,100.0,96.375999,51.306599,52.773998,155.868500,794.815186,29.411301,157.107101,271.072388,1044.551880,12.079600,11.3235,403.578491,77.590897,78.996300,12.0,12.2162,1.3872,0.1787,35.482800,36.688202,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6509,-4.5318,-1.7957,0.8014,1.9603,1816.0,13136.0,70.0,10.0,18.225500,0.0007,65.263298,60.854301,0.0,390.136688,0.0007,324.688995,326.931396,0.0,10053.0,27616.0,0,0,0,0,1
11,0.0998,1.3647,404.178711,1040.812500,35.200001,9.860600,154.128006,35.662102,12.729400,12.254300,100.0,96.375999,51.862701,51.881401,160.064697,795.684814,29.029900,149.162598,262.679993,1032.433472,12.637200,11.9644,403.782898,77.458397,79.003899,12.0,12.2238,1.3879,0.1772,35.467499,36.673000,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.8274,-3.7878,-1.7647,-0.0491,1.2151,1296.0,13408.0,70.0,10.0,18.149599,0.0002,64.431396,62.554298,0.0,383.825287,-0.0014,318.196594,323.784698,0.0,10053.0,27617.0,0,0,0,0,1
13,0.0998,1.3642,404.178711,1040.812500,35.200001,9.895300,151.148804,35.660900,12.833900,12.254300,100.0,96.375999,51.898499,51.835602,163.879395,815.254211,29.029900,151.148804,264.587402,1033.390137,12.674800,12.5061,404.038513,77.449501,79.003899,12.0,12.2238,1.3734,0.1764,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7151,-4.8982,-2.1363,0.1604,1.8099,1168.0,13440.0,70.0,10.0,18.117001,0.0002,64.214401,62.789398,0.0,381.944397,0.0005,318.612488,322.645386,0.0,10053.0,27618.0,0,0,0,0,1
21,0.0998,1.3495,404.178711,1040.812500,35.200001,9.995600,151.148804,35.624298,13.271600,12.254300,100.0,96.375999,52.111801,51.835602,166.740402,816.559021,28.457600,153.134705,265.159607,1034.027954,12.781800,12.7426,404.856415,77.441299,78.996300,12.0,12.2238,1.3635,0.1756,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.8301,-3.7905,-1.7644,-0.0515,1.2212,776.0,13472.0,70.0,10.0,17.896400,0.0000,63.375301,63.078701,0.0,376.211700,0.0001,315.393494,318.540192,0.0,10052.0,27618.0,0,0,0,0,1
25,0.0998,1.3413,404.178711,1040.812500,35.200001,10.034100,148.169296,35.603699,13.509800,12.254300,100.0,96.375999,52.238201,51.835602,161.781296,815.689087,29.029900,152.141693,266.303986,1033.390137,12.811700,12.7426,405.418701,77.359398,79.003899,12.0,12.2238,1.3635,0.1779,35.482800,36.657700,28.021200,1.0,0.0,1.0,0.0,791.0,0.0,11.7131,-3.8701,-1.5639,0.7177,1.6119,632.0,13456.0,70.0,10.0,17.842199,0.0000,63.035301,62.970200,0.0,375.705292,0.0009,314.290314,317.093414,0.0,10053.0,27618.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181785,0.0305,1.3176,392.330292,1115.077148,35.200001,83.772003,2541.829834,35.717800,40.041801,38.980099,0.0,-1.852400,58.491402,59.137001,187.149002,865.699829,1598.777710,2539.563965,313.796906,1115.668457,16.862101,16.8930,390.185913,42.717098,42.375198,12.0,11.6364,1.2651,0.7463,35.208099,35.574299,28.019501,1.0,0.0,1.0,0.0,791.0,0.0,11.6345,-4.9013,-2.1021,0.3981,1.8172,3456.0,4208.0,70.0,10.0,28.577101,0.0008,11.335300,21.665199,0.0,343.460602,0.0034,330.168488,329.282410,0.0,10026.0,27590.0,0,0,0,0,1
181786,0.0305,1.3254,392.330292,1115.077148,35.200001,83.828903,2539.563965,35.737202,39.957100,39.918499,0.0,

time: 1min 29s


# **A1 N**

# **5%**

In [11]:
picklename = '/drive/My Drive/data/AAN_new/A1_N/reset_Error_whole_x_test_5percent.pickle'
percent = 0.05
target = 2
max = 500 
target2 = 4
max2 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
174156
에러 O
7644


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
17,0.0998,1.3564,541.599731,1040.812500,35.200001,9.950200,151.148804,35.641499,13.007800,12.254300,100.0,96.375999,52.183102,51.835602,164.070099,810.470520,29.029900,154.128006,263.633698,1028.925415,12.9254,12.5214,404.396393,77.488098,79.003899,12.0,12.2238,1.3628,0.1749,35.467499,36.673000,28.019501,1.0,0.0,1.0,0.0,792.0,0.0,11.8188,-4.4556,-2.1329,-0.3630,1.3600,952.0,13472.0,70.0,10.0,17.968800,0.0002,63.722500,63.024502,0.0,379.032806,0.0008,315.285004,319.968903,0.0,10053.0,27617.0,0,0,0,0,1
21,0.0998,1.3495,541.599731,1040.812500,35.200001,9.995600,151.148804,35.624298,13.271600,12.254300,100.0,96.375999,52.111801,51.835602,166.740402,816.559021,28.457600,153.134705,265.159607,1034.027954,12.7818,12.7426,404.856415,77.441299,78.996300,12.0,12.2238,1.3635,0.1756,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.8301,-3.7905,-1.7644,-0.0515,1.2212,776.0,13472.0,70.0,10.0,17.896400,0.0000,63.375301,63.078701,0.0,376.211700,0.0001,315.393494,318.540192,0.0,10052.0,27618.0,0,0,0,0,1
22,0.0998,1.3453,541.599731,1040.812500,35.200001,10.016100,148.169296,35.613602,13.402800,12.254300,100.0,96.375999,52.125500,51.835602,162.353500,819.168091,29.792700,148.169296,265.159607,1034.346802,12.7791,12.7426,405.009796,77.404602,78.996300,12.0,12.2238,1.3628,0.1779,35.467499,36.657700,28.021999,1.0,0.0,1.0,0.0,790.0,0.0,11.8231,-4.4297,-2.1198,-0.3710,1.3449,736.0,13472.0,70.0,10.0,17.896400,0.0000,63.281300,63.024502,0.0,377.658386,-0.0009,312.698914,318.232788,0.0,10052.0,27616.0,0,0,0,0,1
29,0.0998,1.3393,541.599731,1040.812500,35.200001,10.062200,151.148804,35.598801,13.645300,13.299600,100.0,96.375999,52.189201,52.949501,165.405197,812.645081,29.029900,158.100403,266.685486,1036.898193,12.7279,12.7426,405.674286,77.301598,79.011497,12.0,12.2238,1.3620,0.1741,35.467499,36.657700,28.021999,1.0,0.0,1.0,0.0,792.0,0.0,11.6479,-4.5810,-1.8256,0.7598,1.9617,528.0,13424.0,70.0,10.0,17.780701,0.0000,62.745899,62.897900,0.0,375.036194,0.0007,312.174500,316.008392,0.0,10053.0,27618.0,0,0,0,0,1
97,0.0998,1.6259,541.599731,1040.812500,35.200001,13.328700,265.354187,36.315201,15.680200,16.625999,0.0,43.328899,52.369900,52.072102,155.296295,794.815186,66.223099,294.154114,264.396606,1033.071167,11.8357,11.4380,407.514496,80.929398,81.582603,12.0,12.2238,1.3887,0.3152,35.498001,36.550900,28.026400,1.0,0.0,1.0,0.0,792.0,0.0,11.8379,-3.8315,-1.8059,-0.1340,1.1907,16.0,16864.0,70.0,10.0,16.503901,-0.0002,79.875603,78.269699,0.0,483.904785,-0.0021,403.826691,393.952606,0.0,10053.0,27618.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181719,0.0305,1.0849,549.262512,1115.077148,35.200001,80.738098,2427.511719,35.136002,37.417999,38.133202,0.0,-1.852400,58.033501,59.030201,171.318100,833.518982,1606.597900,2487.145508,311.889587,1109.928101,15.2843,14.1693,390.185913,42.202599,43.328899,12.0,11.6287,1.3322,0.8165,35.223400,35.589600,28.021999,1.0,0.0,1.0,0.0,792.0,0.0,11.6335,-4.7394,-1.9796,0.5630,1.8353,11832.0,272.0,70.0,10.0,19.994200,-0.0004,-0.007200,2.369000,0.0,255.768997,-0.0007,255.588104,267.487701,0.0,10029.0,27596.0,0,0,0,0,1
181724,0.0305,1.0702,549.262512,1115.077148,35.200001,80.624702,2521.134521,35.099400,38.393398,36.378502,0.0,-1.852400,

time: 25 s


10

In [12]:
picklename = '/drive/My Drive/data/AAN_new/A1_N/reset_Error_whole_x_test_10percent.pickle'
percent = 0.1
target = 2
max = 500 
target2 = 4
max2 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
166512
에러 O
15288


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,0.0998,1.3901,541.599731,1040.812500,35.200001,9.672400,149.162598,35.725800,12.025100,11.186200,100.000000,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.1733,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,1
10,0.0998,1.3688,541.599731,1040.812500,35.200001,9.822000,154.128006,35.672401,12.653100,12.254300,100.000000,96.375999,51.756500,51.843300,156.440704,796.119690,28.648399,151.148804,264.015198,1034.346802,12.529000,11.3235,403.936310,77.488800,79.003899,12.0,12.2238,1.3864,0.1810,35.498001,36.688202,28.018600,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-3.8636,-1.5659,0.7002,1.5998,1368.0,13392.0,70.0,10.0,18.196600,0.0002,64.597801,62.427601,0.0,384.494385,-0.0001,317.852997,324.634613,0.0,10053.0,27617.0,0,0,0,0,1
12,0.0998,1.3647,541.599731,1040.812500,35.200001,9.848000,151.148804,35.662102,12.773500,12.254300,100.000000,96.375999,51.990002,51.866100,165.023804,803.947510,29.029900,151.148804,264.587402,1030.200928,12.791100,12.4832,403.987396,77.439499,79.003899,12.0,12.2238,1.3834,0.1779,35.467499,36.688202,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.8210,-4.4473,-2.1370,-0.3822,1.3351,1232.0,13424.0,70.0,10.0,18.095301,0.0002,64.308403,62.662800,0.0,382.613586,0.0012,317.889191,323.187897,0.0,10053.0,27616.0,0,0,0,0,1
33,0.0998,1.3337,541.599731,1040.812500,35.200001,10.093800,151.148804,35.584599,13.985000,13.246200,100.000000,96.383698,51.885101,52.842701,166.358902,816.559021,30.174200,151.148804,271.453888,1044.870850,12.322200,12.7426,406.083191,77.221901,78.996300,12.0,12.2238,1.3635,0.1764,35.467499,36.642399,28.021200,1.0,0.0,1.0,0.0,793.0,0.0,11.7378,-4.8530,-2.1606,0.0231,1.7362,424.0,13392.0,70.0,10.0,17.719200,0.0000,62.521702,62.735100,0.0,373.498993,-0.0013,311.686188,314.543488,0.0,10053.0,27616.0,0,0,0,0,1
56,0.0998,1.3682,541.599731,1040.812500,35.200001,10.415600,156.114197,35.670898,15.338800,14.276100,52.180099,96.002197,51.595299,53.010601,166.168198,819.603088,29.601999,154.128006,271.644592,1045.827515,11.619400,12.7426,407.514496,77.005798,78.286697,12.0,12.2238,1.3612,0.1764,35.482800,36.611900,28.026400,1.0,0.0,1.0,0.0,792.0,0.0,11.7710,-3.7147,-1.6243,0.3295,1.3603,136.0,13744.0,70.0,10.0,17.372000,-0.0004,65.574402,63.458500,0.0,413.628510,0.0012,348.180695,328.938812,0.0,10051.0,27618.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181755,0.0305,1.3438,549.262512,1115.077148,35.200001,82.044098,2529.593994,35.783401,38.813202,37.805199,0.000000,-1.852400,58.336800,59.030201,180.091797,843.521179,1535.835327,2503.611328,312.271088,1111.522827,16.319799,15.5884,389.674713,41.704300,42.375198,12.0,11.6287,1.2910,0.8165,35.238602,35.589600,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.7163,-4.0044,-1.7419,0.4276,1.4117,10416.0,-208.0,70.0,10.0,29.676600,-0.0002,-0.007200,0.108500,0.0,353.660309,0.0008,354.202789,338.198090,0.0,10026.0,27587.0,0,0,0,0,1
181759,0.0305,1.3600,549.262512,1115.077148,35.200001,82.321404,2509.200439,35

time: 49.4 s


15

In [13]:
picklename = '/drive/My Drive/data/AAN_new/A1_N/reset_Error_whole_x_test_15percent.pickle'
percent = 0.15
target = 2
max = 500 
target2 = 4
max2 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
158867
에러 O
22933


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
2,0.0998,1.3837,541.599731,1040.812500,35.200001,9.704100,152.141693,35.709801,12.180500,11.186200,100.0,96.375999,51.424599,52.773998,159.110992,793.510376,29.029900,151.148804,269.927887,1042.000732,12.198100,11.3235,403.782898,77.600304,78.996300,12.0,12.2238,1.3872,0.1749,35.482800,36.688202,28.0247,1.0,0.0,1.0,0.0,792.0,0.0,11.8218,-4.4148,-2.1085,-0.3602,1.3557,2016.0,12976.0,70.0,10.0,18.268999,0.0007,65.357399,60.004299,0.0,391.167511,0.0008,325.900604,328.540894,0.0,10053.0,27616.0,0,0,0,0,1
8,0.0998,1.3723,541.599731,1040.812500,35.200001,9.820800,153.134705,35.681198,12.494100,11.506700,100.0,96.383698,51.688801,51.866100,157.394394,799.598572,29.029900,154.128006,264.587402,1034.665771,12.484600,11.3235,403.782898,77.522003,78.996300,12.0,12.2238,1.3864,0.1756,35.498001,36.673000,28.0221,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-4.8966,-2.1376,0.1598,1.8016,1512.0,13328.0,70.0,10.0,18.236401,0.0005,64.822098,62.011700,0.0,386.465607,0.0014,320.999695,324.924103,0.0,10053.0,27617.0,0,0,0,0,1
14,0.0998,1.3636,541.599731,1040.812500,35.200001,9.913000,149.162598,35.659401,12.893300,12.254300,100.0,96.375999,51.928200,51.835602,163.497894,812.645081,29.792700,151.148804,267.066986,1033.390137,12.700200,12.5061,404.191895,77.488701,79.003899,12.0,12.2238,1.3681,0.1764,35.467499,36.673000,28.0273,1.0,0.0,1.0,0.0,792.0,0.0,11.6509,-4.5008,-1.7731,0.8137,1.9504,1112.0,13456.0,70.0,10.0,18.080900,0.0002,64.091400,62.843601,0.0,381.474213,0.0001,316.641388,321.994385,0.0,10053.0,27614.0,0,0,0,0,1
16,0.0998,1.3589,541.599731,1040.812500,35.200001,9.938600,151.148804,35.647598,13.007000,12.254300,100.0,96.375999,51.917400,51.835602,162.925705,815.254211,29.601999,151.148804,261.917114,1034.665771,12.647800,12.5061,404.243011,77.500000,78.996300,12.0,12.2162,1.3643,0.1779,35.482800,36.657700,28.0203,1.0,0.0,1.0,0.0,792.0,0.0,11.8299,-3.7922,-1.7679,-0.0526,1.2261,1008.0,13472.0,70.0,10.0,18.015800,0.0002,63.845501,62.970200,0.0,378.237091,-0.0019,316.550903,320.547607,0.0,10053.0,27613.0,0,0,0,0,1
27,0.0998,1.3386,541.599731,1040.812500,35.200001,10.037200,149.162598,35.596802,13.605600,12.254300,100.0,96.375999,52.219101,51.835602,163.307205,807.861389,29.411301,153.134705,264.968903,1034.984619,12.763000,12.7426,405.469788,77.321404,78.996300,12.0,12.2314,1.3628,0.1779,35.482800,36.657700,28.0247,1.0,0.0,1.0,0.0,792.0,0.0,11.8282,-4.3655,-2.0912,-0.3608,1.3324,584.0,13440.0,70.0,10.0,17.806000,-0.0002,62.868900,62.970200,0.0,377.622314,-0.0014,313.187195,317.328613,0.0,10052.0,27618.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181756,0.0305,1.3536,549.262512,1115.077148,35.200001,82.111198,2484.275391,35.807800,38.944801,37.789902,0.0,-1.852400,58.284000,59.030201,177.040100,849.609375,1548.232910,2499.381592,312.080292,1113.117310,16.278601,15.5884,389.674713,41.778301,42.375198,12.0,11.6287,1.2918,0.8142,35.208099,35.574299,28.0194,1.0,0.0,1.0,0.0,792.0,0.0,11.8235,-4.0347,-1.9833,-0.3084,1.0793,10144.0,-208.0,70.0,10.0,29.720100,-0.0002,-0.007200,0.108500,0.0,356.119812,-0.0011,354.745392,339.753387,0.0,10026.0,27590.0,0,0,0,0,1
181776,0.0305,1.3292,549.262512,1115.077148,35.200001,83.361603,2548.778809,35.746799,39.310600,38.987701,0.0,-1.852

time: 1min 10s


30

In [14]:
picklename = '/drive/My Drive/data/AAN_new/A1_N/reset_Error_whole_x_test_30percent.pickle'
percent = 0.3
target = 2
max = 500 
target2 = 4
max2 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
135934
에러 O
45866


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,0.0998,1.3901,541.599731,1040.812500,35.200001,9.672400,149.162598,35.725800,12.025100,11.186200,100.0,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.1733,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,1
1,0.0998,1.3840,541.599731,1040.812500,35.200001,9.696700,150.155502,35.710499,12.156200,11.186200,100.0,96.375999,51.393002,52.773998,155.487000,799.163818,29.029900,149.162598,269.737213,1042.957397,12.136500,11.3235,403.680695,77.599503,79.003899,12.0,12.2238,1.3864,0.1779,35.498001,36.688202,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8256,-3.7916,-1.7599,-0.0161,1.2254,2120.0,12896.0,70.0,10.0,18.236401,0.0005,65.407997,59.570301,0.0,394.223785,-0.0027,325.882507,329.842987,0.0,10053.0,27617.0,0,0,0,0,1
3,0.0998,1.3854,541.599731,1040.812500,35.200001,9.715200,154.128006,35.714001,12.247300,11.186200,100.0,96.375999,51.406399,52.773998,156.059296,801.773193,30.174200,151.148804,271.263092,1042.319580,12.170300,11.3235,403.782898,77.603798,79.003899,12.0,12.2238,1.3879,0.1772,35.482800,36.688202,28.024599,1.0,0.0,1.0,0.0,792.0,0.0,11.7147,-4.9002,-2.1456,0.1541,1.8032,1912.0,13056.0,70.0,10.0,18.243601,0.0005,65.328400,60.474499,0.0,390.606903,-0.0001,325.159088,328.215393,0.0,10053.0,27618.0,0,0,0,0,1
5,0.0998,1.3804,541.599731,1040.812500,35.200001,9.762300,151.148804,35.701401,12.329600,11.186200,100.0,96.375999,51.305099,52.773998,158.920303,794.380188,29.029900,153.134705,269.165009,1044.233032,12.117700,11.3159,403.731812,77.574402,79.003899,12.0,12.2238,1.3872,0.1764,35.482800,36.673000,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.7141,-3.8737,-1.5734,0.6949,1.5980,1720.0,13184.0,70.0,10.0,18.261700,0.0005,65.191002,61.216000,0.0,388.798492,-0.0013,323.585785,326.913300,0.0,10053.0,27617.0,0,0,0,0,1
6,0.0998,1.3785,541.599731,1040.812500,35.200001,9.779400,151.148804,35.696800,12.416400,11.186200,100.0,96.375999,51.425301,52.751202,157.966599,801.338196,29.601999,151.148804,269.546509,1041.043823,12.216300,11.3235,403.731812,77.561401,78.996300,12.0,12.2238,1.3872,0.1794,35.467499,36.688202,28.027201,1.0,0.0,1.0,0.0,793.0,0.0,11.8285,-3.7991,-1.7702,-0.0452,1.2192,1640.0,13248.0,70.0,10.0,18.229200,0.0007,65.039101,61.505402,0.0,384.928406,0.0014,320.384888,325.575104,0.0,10053.0,27614.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181788,0.0305,1.3252,549.262512,1115.077148,35.200001,83.934700,2575.516846,35.736801,39.887299,40.223701,0.0,-1.852400,58.534100,59.144600,189.056396,870.048584,1612.892090,2561.316895,312.271088,1113.117310,16.965000,16.8930,390.594788,42.752201,43.595901,12.0,11.6287,1.2689,0.7265,35.208099,35.559101,28.022900,1.0,0.0,1.0,0.0,791.0,0.0,11.8327,-4.1613,-2.0838,-0.4562,1.0633,4944.0,3248.0,70.0,10.0,28.125000,-0.0002,6.083600,16.728100,0.0,338.849091,0.0003,334.671600,330.313202,0.0,10026.0,27587.0,0,0,0,0,1
181790,0.0305,1.3263,549.262512,1115.077148,35.200001,84.040100,2560.410645,35.739498,39.837002,40.238899,0.0,

time: 2min 15s


# **A2_N**

5

In [15]:
picklename = '/drive/My Drive/data/AAN_new/A2_N/reset_Error_whole_x_test_5percent.pickle'
percent = 0.05
target = 3
max = 1100 
target2 = 4
max2 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
174156
에러 O
7644


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
31,0.0998,1.3341,404.178711,1207.342529,35.200001,10.088100,158.100403,35.585800,13.808550,13.208000,100.0,96.375999,52.152401,52.941898,163.593246,814.167053,30.078850,153.134857,269.069641,1038.333252,12.636650,12.742600,405.980988,77.256897,78.996300,12.0,12.2238,1.3612,0.17640,35.475151,36.642399,28.025499,1.0,0.0,1.0,0.0,791.0,0.0,11.8118,-3.7390,-1.7139,0.0543,1.2467,472.0,13408.0,70.0,10.0,17.751699,0.0000,62.623001,62.789398,0.0,372.992615,0.0018,310.221405,315.520111,0.0,10053.0,27614.0,0,0,0,0,1
38,0.0998,1.3233,404.178711,1207.342529,35.200001,10.135700,151.148804,35.558701,14.334400,13.200400,100.0,96.375999,51.933998,53.010601,163.116501,817.863525,29.601999,147.176407,271.263092,1042.319580,12.303500,12.742600,406.134308,77.072502,79.003899,12.0,12.2238,1.3620,0.17640,35.482800,36.642399,28.026400,1.0,0.0,1.0,0.0,793.0,0.0,11.7392,-4.8500,-2.1654,0.0118,1.7303,328.0,13344.0,70.0,10.0,17.657700,0.0000,62.283001,62.554298,0.0,371.961914,0.0008,310.040497,313.060699,0.0,10053.0,27618.0,0,0,0,0,1
82,0.0998,1.5659,404.178711,1207.342529,35.200001,12.051400,214.706696,36.165199,16.194700,15.580700,0.0,48.127701,51.971100,52.072102,159.683197,793.510376,47.721901,231.589096,266.113190,1035.941284,11.675400,11.438000,407.207794,79.029999,79.583702,12.0,12.2238,1.3902,0.24730,35.513302,36.581402,28.024700,1.0,0.0,1.0,0.0,793.0,0.0,11.8379,-3.8452,-1.8217,-0.1540,1.1914,32.0,16112.0,70.0,10.0,16.909000,0.0000,77.293098,74.435799,0.0,471.281799,0.0012,393.120697,378.074310,0.0,10053.0,27618.0,0,0,0,0,1
89,0.0998,1.6013,404.178711,1207.342529,35.200001,12.633600,439.145508,36.253700,13.835000,13.574200,100.0,40.681499,52.127300,52.072102,156.440704,792.640625,81.100502,452.055786,265.922485,1035.622437,11.728900,11.438000,407.309998,79.891800,80.911201,12.0,12.2238,1.3887,0.39760,35.498001,36.566200,28.028999,1.0,0.0,1.0,0.0,793.0,0.0,11.6979,-4.8835,-2.1021,0.2527,1.8449,24.0,16528.0,70.0,10.0,16.731800,0.0000,78.725403,76.533600,0.0,477.430603,0.0008,398.763000,386.700714,0.0,10053.0,27619.0,0,0,0,0,1
100,0.0998,1.6399,404.178711,1207.342529,35.200001,13.613400,391.477112,36.350300,13.610800,14.970400,100.0,37.858601,52.301102,52.072102,154.819443,792.640625,93.307503,438.152588,267.448395,1038.173706,11.655000,11.438000,407.514496,81.415001,82.879601,12.0,12.2314,1.3879,0.42660,35.498001,36.535599,28.020300,1.0,0.0,1.0,0.0,791.0,0.0,11.6516,-4.4455,-1.7477,0.8238,1.9240,16.0,16816.0,70.0,10.0,16.438801,0.0010,79.340302,78.956902,0.0,459.725800,0.0049,378.906311,386.989990,0.0,10053.0,27619.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181758,0.0305,1.3574,392.330292,1226.584961,35.200001,82.253601,2499.381592,35.817299,38.183899,38.972500,0.0,-1.852400,58.367901,59.030201,178.852051,847.869873,1606.597900,2556.331787,312.271088,1111.522827,16.404600,15.588400,389.725800,41.866600,42.375198,12.0,11.6287,1.2910,0.76385,35.208099,35.566700,28.023800,1.0,0.0,1.0,0.0,791.0,0.0,11.6954,-5.0605,-2.2912,0.0212,1.7023,9160.0,-208.0,70.0,10.0,29.756201,-0.0002,-0.007200,0.072400,0.0,357.928192,-0.0012,356.644196,342.556396,0.0,10026.0,27595.0,0,0,0,0,1
181770,0.0305,1.3777,392.330292,1226.584961,35.200001,83.108498,2532.766113,35.868099,39.120499

time: 23.9 s


10

In [16]:
picklename = '/drive/My Drive/data/AAN_new/A2_N/reset_Error_whole_x_test_10percent.pickle'
percent = 0.1
target = 3
max = 1100 
target2 = 4
max2 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
166512
에러 O
15288


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
1,0.0998,1.3840,404.178711,1207.342529,35.200001,9.696700,150.155502,35.710499,12.156200,11.186200,100.0,96.375999,51.393002,52.773998,155.487000,799.163818,29.029900,149.162598,269.737213,1042.957397,12.136500,11.3235,403.680695,77.599503,79.003899,12.0,12.2238,1.3864,0.1779,35.498001,36.688202,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8256,-3.7916,-1.7599,-0.0161,1.2254,2120.0,12896.0,70.0,10.0,18.236401,0.0005,65.407997,59.570301,0.0,394.223785,-0.0027,325.882507,329.842987,0.0,10053.0,27617.0,0,0,0,0,1
18,0.0998,1.3528,404.178711,1207.342529,35.200001,9.942000,155.120895,35.632301,13.115400,12.254300,100.0,96.375999,52.110100,51.835602,163.879395,813.079773,28.457600,151.148804,264.396606,1031.795654,12.832900,12.7350,404.549713,77.473000,78.996300,12.0,12.2238,1.3628,0.1764,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7140,-4.8897,-2.1313,0.1738,1.8179,904.0,13472.0,70.0,10.0,17.932600,0.0003,63.621201,63.024502,0.0,379.701996,0.0018,315.936005,319.824188,0.0,10052.0,27621.0,0,0,0,0,1
19,0.0998,1.3480,404.178711,1207.342529,35.200001,9.960500,151.148804,35.620499,13.224900,12.254300,100.0,96.375999,52.092602,51.835602,165.214493,812.645081,28.076099,148.169296,264.778107,1033.071167,12.793500,12.7426,404.651886,77.443100,78.996300,12.0,12.2238,1.3628,0.1764,35.467499,36.673000,28.023800,1.0,0.0,1.0,0.0,791.0,0.0,11.6508,-4.4918,-1.7711,0.8150,1.9531,856.0,13472.0,70.0,10.0,17.968800,0.0000,63.556099,63.024502,0.0,377.857300,0.0003,315.936005,320.113586,0.0,10051.0,27613.0,0,0,0,0,1
33,0.0998,1.3337,404.178711,1207.342529,35.200001,10.093800,151.148804,35.584599,13.985000,13.246200,100.0,96.383698,51.885101,52.842701,166.358902,816.559021,30.174200,151.148804,271.453888,1044.870850,12.322200,12.7426,406.083191,77.221901,78.996300,12.0,12.2238,1.3635,0.1764,35.467499,36.642399,28.021200,1.0,0.0,1.0,0.0,793.0,0.0,11.7378,-4.8530,-2.1606,0.0231,1.7362,424.0,13392.0,70.0,10.0,17.719200,0.0000,62.521702,62.735100,0.0,373.498993,-0.0013,311.686188,314.543488,0.0,10053.0,27616.0,0,0,0,0,1
37,0.0998,1.3247,404.178711,1207.342529,35.200001,10.129800,157.107101,35.562099,14.238300,13.200400,100.0,96.375999,51.915798,53.010601,166.168198,814.384399,28.266899,149.162598,269.927887,1042.957397,12.301200,12.7426,406.236603,77.106796,79.003899,12.0,12.2238,1.3643,0.1756,35.467499,36.642399,28.022900,1.0,0.0,1.0,0.0,791.0,0.0,11.8365,-4.2743,-2.0594,-0.3680,1.2860,344.0,13360.0,70.0,10.0,17.686600,0.0000,62.333599,62.590401,0.0,373.119293,0.0016,310.221405,313.350006,0.0,10053.0,27619.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181750,0.0305,1.3269,392.330292,1226.584961,35.200001,81.690697,2477.175293,35.730549,37.648300,38.736000,0.0,-1.852400,58.248501,59.030201,174.751297,837.867676,1602.210938,2557.842529,312.080292,1113.117310,16.128401,15.5884,389.725800,41.459099,42.382801,12.0,11.6287,1.2903,0.7677,35.223400,35.589600,28.024700,1.0,0.0,1.0,0.0,793.0,0.0,11.7114,-4.0061,-1.7287,0.4611,1.4421,13408.0,-208.0,70.0,10.0,28.070700,0.0000,-0.007200,0.162800,0.0,353.352814,0.0045,351.978394,331.217499,0.0,10026.0,27587.0,0,0,0,0,1
181754,0.0305,1.3342,392.330292,1226.584961,35.200001,81.949203,2531.104492,35.759399,38.646301,37.843300,0.0,

time: 47.4 s


15

In [17]:
picklename = '/drive/My Drive/data/AAN_new/A2_N/reset_Error_whole_x_test_15percent.pickle'
percent = 0.15
target = 3
max = 1100 
target2 = 4
max2 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
158867
에러 O
22933


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
4,0.0998,1.3839,404.178711,1207.342529,35.200001,9.748800,149.162598,35.710098,12.209800,11.186200,100.0,96.375999,51.306599,52.773998,155.868500,794.815186,29.411301,157.107101,271.072388,1044.551880,12.079600,11.3235,403.578491,77.590897,78.996300,12.0,12.2162,1.3872,0.1787,35.482800,36.688202,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6509,-4.5318,-1.7957,0.8014,1.9603,1816.0,13136.0,70.0,10.0,18.225500,0.0007,65.263298,60.854301,0.0,390.136688,0.0007,324.688995,326.931396,0.0,10053.0,27616.0,0,0,0,0,1
16,0.0998,1.3589,404.178711,1207.342529,35.200001,9.938600,151.148804,35.647598,13.007000,12.254300,100.0,96.375999,51.917400,51.835602,162.925705,815.254211,29.601999,151.148804,261.917114,1034.665771,12.647800,12.5061,404.243011,77.500000,78.996300,12.0,12.2162,1.3643,0.1779,35.482800,36.657700,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.8299,-3.7922,-1.7679,-0.0526,1.2261,1008.0,13472.0,70.0,10.0,18.015800,0.0002,63.845501,62.970200,0.0,378.237091,-0.0019,316.550903,320.547607,0.0,10053.0,27613.0,0,0,0,0,1
18,0.0998,1.3528,404.178711,1207.342529,35.200001,9.942000,155.120895,35.632301,13.115400,12.254300,100.0,96.375999,52.110100,51.835602,163.879395,813.079773,28.457600,151.148804,264.396606,1031.795654,12.832900,12.7350,404.549713,77.473000,78.996300,12.0,12.2238,1.3628,0.1764,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7140,-4.8897,-2.1313,0.1738,1.8179,904.0,13472.0,70.0,10.0,17.932600,0.0003,63.621201,63.024502,0.0,379.701996,0.0018,315.936005,319.824188,0.0,10052.0,27621.0,0,0,0,0,1
36,0.0998,1.3274,404.178711,1207.342529,35.200001,10.141400,151.148804,35.569000,14.150000,13.200400,100.0,96.375999,51.875900,53.010601,164.642303,815.689087,28.648399,151.148804,270.309387,1044.233032,12.277500,12.7426,406.185394,77.146103,78.996300,12.0,12.2238,1.3628,0.1764,35.467499,36.642399,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.8028,-3.7371,-1.6995,0.1108,1.2659,368.0,13360.0,70.0,10.0,17.690201,-0.0002,62.376999,62.590401,0.0,372.196899,-0.0006,310.022400,313.621185,0.0,10052.0,27617.0,0,0,0,0,1
43,0.0998,1.3166,404.178711,1207.342529,35.200001,10.160700,153.134705,35.541901,14.492900,13.208000,100.0,96.375999,51.810101,53.010601,164.070099,808.731201,29.411301,154.128006,271.072388,1043.914062,12.122300,12.7426,406.492188,76.955597,78.996300,12.0,12.2238,1.3612,0.1772,35.467499,36.627201,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7439,-4.8497,-2.1730,-0.0176,1.7018,256.0,13312.0,70.0,10.0,17.559999,0.0000,62.116600,62.355301,0.0,371.708588,0.0009,310.167114,311.450989,0.0,10052.0,27616.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181754,0.0305,1.3342,392.330292,1226.584961,35.200001,81.949203,2531.104492,35.759399,38.646301,37.843300,0.0,-1.852400,58.192650,59.030201,177.898407,845.478088,1528.968750,2506.934814,311.984985,1114.871338,16.128250,15.5884,389.623596,41.628201,42.382801,12.0,11.6249,1.2903,0.8043,35.223400,35.574299,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.6376,-4.5837,-1.9032,0.6405,1.7893,10968.0,-208.0,70.0,10.0,29.615200,0.0000,-0.007200,0.108500,0.0,354.076294,0.0033,352.448700,336.480011,0.0,10026.0,27596.0,0,0,0,0,1
181757,0.0305,1.3577,392.330292,1226.584961,35.200001,82.164398,2503.611328,35.818100,38.870701,38.865700,0.

time: 1min 8s


30

In [18]:
picklename = '/drive/My Drive/data/AAN_new/A2_N/reset_Error_whole_x_test_30percent.pickle'
percent = 0.3
target = 3
max = 1100 
target2 = 4
max2 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
135934
에러 O
45866


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,0.0998,1.3901,404.178711,1207.342529,35.200001,9.672400,149.162598,35.725800,12.025100,11.186200,100.0,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.1733,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,1
1,0.0998,1.3840,404.178711,1207.342529,35.200001,9.696700,150.155502,35.710499,12.156200,11.186200,100.0,96.375999,51.393002,52.773998,155.487000,799.163818,29.029900,149.162598,269.737213,1042.957397,12.136500,11.3235,403.680695,77.599503,79.003899,12.0,12.2238,1.3864,0.1779,35.498001,36.688202,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8256,-3.7916,-1.7599,-0.0161,1.2254,2120.0,12896.0,70.0,10.0,18.236401,0.0005,65.407997,59.570301,0.0,394.223785,-0.0027,325.882507,329.842987,0.0,10053.0,27617.0,0,0,0,0,1
10,0.0998,1.3688,404.178711,1207.342529,35.200001,9.822000,154.128006,35.672401,12.653100,12.254300,100.0,96.375999,51.756500,51.843300,156.440704,796.119690,28.648399,151.148804,264.015198,1034.346802,12.529000,11.3235,403.936310,77.488800,79.003899,12.0,12.2238,1.3864,0.1810,35.498001,36.688202,28.018600,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-3.8636,-1.5659,0.7002,1.5998,1368.0,13392.0,70.0,10.0,18.196600,0.0002,64.597801,62.427601,0.0,384.494385,-0.0001,317.852997,324.634613,0.0,10053.0,27617.0,0,0,0,0,1
13,0.0998,1.3642,404.178711,1207.342529,35.200001,9.895300,151.148804,35.660900,12.833900,12.254300,100.0,96.375999,51.898499,51.835602,163.879395,815.254211,29.029900,151.148804,264.587402,1033.390137,12.674800,12.5061,404.038513,77.449501,79.003899,12.0,12.2238,1.3734,0.1764,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7151,-4.8982,-2.1363,0.1604,1.8099,1168.0,13440.0,70.0,10.0,18.117001,0.0002,64.214401,62.789398,0.0,381.944397,0.0005,318.612488,322.645386,0.0,10053.0,27618.0,0,0,0,0,1
15,0.0998,1.3621,404.178711,1207.342529,35.200001,9.928200,151.148804,35.655602,12.887500,12.254300,100.0,96.391296,51.783699,51.835602,165.023804,811.775330,29.029900,155.120895,265.350311,1037.536011,12.539500,12.5061,404.345215,77.503799,79.003899,12.0,12.2314,1.3651,0.1764,35.467499,36.673000,28.022100,1.0,0.0,1.0,0.0,791.0,0.0,11.7151,-3.8580,-1.5607,0.6961,1.6043,1056.0,13472.0,70.0,10.0,18.037500,0.0002,63.975700,62.897900,0.0,379.195587,0.0013,315.086090,321.849701,0.0,10053.0,27614.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181782,0.0305,1.3042,392.330292,1226.584961,35.200001,83.573097,2553.310547,35.684200,39.717701,38.980099,0.0,-1.852400,58.567402,59.030201,181.427002,852.653503,1594.390869,2546.210693,310.936005,1110.885010,16.923100,16.8930,390.288086,42.505600,42.375198,12.0,11.6287,1.2659,0.7478,35.208099,35.559101,28.022100,1.0,0.0,1.0,0.0,791.0,0.0,11.7891,-3.9211,-1.8757,-0.0968,1.1256,2728.0,5424.0,70.0,10.0,28.917101,0.0030,21.115400,27.976700,0.0,346.987091,-0.0002,325.321899,327.094208,0.0,10026.0,27588.0,0,0,0,0,1
181787,0.0305,1.3270,392.330292,1226.584961,35.200001,83.886200,2549.231934,35.741402,39.603100,40.071098,0.

time: 2min 11s


# **A1_A2**

5

In [19]:
picklename = '/drive/My Drive/data/AAN_new/A1_A2/reset_Error_whole_x_test_5percent.pickle'
percent = 0.05
target = 2
max = 500 
target2 = 3
max2 = 1100

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
174156
에러 O
7644


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
15,0.0998,1.3621,541.599731,1207.342529,32.0,9.928200,151.148804,35.655602,12.887500,12.254300,100.0,96.391296,51.783699,51.835602,165.023804,811.775330,29.029900,155.120895,265.350311,1037.536011,12.539500,12.5061,404.345215,77.503799,79.003899,12.0,12.2314,1.3651,0.1764,35.467499,36.673000,28.022100,1.0,0.0,1.0,0.0,791.0,0.0,11.7151,-3.8580,-1.5607,0.6961,1.6043,1056.0,13472.0,70.0,10.0,18.037500,0.0002,63.975700,62.897900,0.0,379.195587,0.0013,315.086090,321.849701,0.0,10053.0,27614.0,0,0,0,0,1
37,0.0998,1.3247,541.599731,1207.342529,32.0,10.129800,157.107101,35.562099,14.238300,13.200400,100.0,96.375999,51.915798,53.010601,166.168198,814.384399,28.266899,149.162598,269.927887,1042.957397,12.301200,12.7426,406.236603,77.106796,79.003899,12.0,12.2238,1.3643,0.1756,35.467499,36.642399,28.022900,1.0,0.0,1.0,0.0,791.0,0.0,11.8365,-4.2743,-2.0594,-0.3680,1.2860,344.0,13360.0,70.0,10.0,17.686600,0.0000,62.333599,62.590401,0.0,373.119293,0.0016,310.221405,313.350006,0.0,10053.0,27619.0,0,0,0,0,1
53,0.0998,1.3393,541.599731,1207.342529,32.0,10.294600,159.093399,35.598801,15.173600,14.276100,100.0,96.375999,51.817402,53.010601,164.260803,813.514587,29.983500,150.155502,270.500214,1041.362915,11.921200,12.7426,407.105499,76.875999,78.332497,12.0,12.2238,1.3620,0.1756,35.467499,36.596699,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7676,-4.7822,-2.1974,-0.1594,1.6047,152.0,13488.0,70.0,10.0,17.419001,-0.0007,63.715302,62.481899,0.0,400.463013,-0.0013,336.950287,322.048615,0.0,10053.0,27612.0,0,0,0,0,1
65,0.0998,1.4480,541.599731,1207.342529,32.0,10.859400,159.093399,35.870399,16.060600,14.550800,0.0,79.629501,51.414700,53.010601,156.250000,797.424316,30.746500,155.120895,271.263092,1046.465454,11.333000,11.4380,407.105499,77.396698,78.286697,12.0,12.2238,1.3895,0.1787,35.482800,36.596699,28.027201,1.0,0.0,1.0,0.0,792.0,0.0,11.6609,-4.2710,-1.6636,0.8832,1.8868,80.0,14656.0,70.0,10.0,17.238100,-0.0002,71.021400,67.437103,0.0,440.917999,0.0007,370.931000,348.958313,0.0,10053.0,27617.0,0,0,0,0,1
66,0.0998,1.4510,541.599731,1207.342529,32.0,10.890100,157.107101,35.877998,16.086599,15.512100,0.0,77.813698,51.472500,53.010601,156.631500,795.249878,31.127899,160.086700,271.072388,1044.551880,11.409700,11.4380,407.105499,77.442001,78.286697,12.0,12.2238,1.3895,0.1772,35.482800,36.581402,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7582,-3.7339,-1.5836,0.4690,1.4385,80.0,14736.0,70.0,10.0,17.234501,-0.0002,71.506104,67.925400,0.0,443.739197,-0.0014,372.631012,351.255096,0.0,10053.0,27619.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181703,0.0305,1.1629,549.262512,1226.584961,32.0,80.938698,2419.146240,35.331001,37.277500,37.583900,0.0,-1.860000,57.817501,59.030201,175.895599,845.695496,1574.554443,2472.039307,311.508087,1113.755127,14.734900,14.1617,391.105988,44.239899,45.816002,12.0,11.6287,1.3483,0.8424,35.238602,35.589600,28.021999,1.0,0.0,1.0,0.0,791.0,0.0,11.7136,-5.0430,-2.3207,-0.1043,1.6378,9408.0,1632.0,70.0,10.0,20.196800,-0.0012,0.506400,9.078400,0.0,273.256714,-0.0012,272.569397,288.483795,0.0,10040.0,27600.0,0,0,0,0,1
181715,0.0305,1.0988,549.262512,1226.584961,32.0,80.779999,2537.751221,35.170700,37.128399,36.599701,0.0,-1.867700,57.851601,59.030201,174.

time: 25.1 s


10

In [20]:
picklename = '/drive/My Drive/data/AAN_new/A1_A2/reset_Error_whole_x_test_10percent.pickle'
percent = 0.1
target = 2
max = 500 
target2 = 3
max2 = 1100

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
166512
에러 O
15288


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
2,0.0998,1.3837,541.599731,1207.342529,32.0,9.704100,152.141693,35.709801,12.180500,11.186200,100.0,96.375999,51.424599,52.773998,159.110992,793.510376,29.029900,151.148804,269.927887,1042.000732,12.198100,11.3235,403.782898,77.600304,78.996300,12.0,12.2238,1.3872,0.1749,35.482800,36.688202,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8218,-4.4148,-2.1085,-0.3602,1.3557,2016.0,12976.0,70.0,10.0,18.268999,0.0007,65.357399,60.004299,0.0,391.167511,0.0008,325.900604,328.540894,0.0,10053.0,27616.0,0,0,0,0,1
8,0.0998,1.3723,541.599731,1207.342529,32.0,9.820800,153.134705,35.681198,12.494100,11.506700,100.0,96.383698,51.688801,51.866100,157.394394,799.598572,29.029900,154.128006,264.587402,1034.665771,12.484600,11.3235,403.782898,77.522003,78.996300,12.0,12.2238,1.3864,0.1756,35.498001,36.673000,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-4.8966,-2.1376,0.1598,1.8016,1512.0,13328.0,70.0,10.0,18.236401,0.0005,64.822098,62.011700,0.0,386.465607,0.0014,320.999695,324.924103,0.0,10053.0,27617.0,0,0,0,0,1
11,0.0998,1.3647,541.599731,1207.342529,32.0,9.860600,154.128006,35.662102,12.729400,12.254300,100.0,96.375999,51.862701,51.881401,160.064697,795.684814,29.029900,149.162598,262.679993,1032.433472,12.637200,11.9644,403.782898,77.458397,79.003899,12.0,12.2238,1.3879,0.1772,35.467499,36.673000,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.8274,-3.7878,-1.7647,-0.0491,1.2151,1296.0,13408.0,70.0,10.0,18.149599,0.0002,64.431396,62.554298,0.0,383.825287,-0.0014,318.196594,323.784698,0.0,10053.0,27617.0,0,0,0,0,1
18,0.0998,1.3528,541.599731,1207.342529,32.0,9.942000,155.120895,35.632301,13.115400,12.254300,100.0,96.375999,52.110100,51.835602,163.879395,813.079773,28.457600,151.148804,264.396606,1031.795654,12.832900,12.7350,404.549713,77.473000,78.996300,12.0,12.2238,1.3628,0.1764,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7140,-4.8897,-2.1313,0.1738,1.8179,904.0,13472.0,70.0,10.0,17.932600,0.0003,63.621201,63.024502,0.0,379.701996,0.0018,315.936005,319.824188,0.0,10052.0,27621.0,0,0,0,0,1
23,0.0998,1.3456,541.599731,1207.342529,32.0,10.017000,146.183105,35.614399,13.348200,12.254300,100.0,96.375999,52.151402,51.835602,166.931198,809.166016,28.648399,155.120895,265.922485,1034.346802,12.779300,12.7426,405.060913,77.401100,78.996300,12.0,12.2238,1.3628,0.1764,35.467499,36.657700,28.019501,1.0,0.0,1.0,0.0,791.0,0.0,11.7213,-4.8861,-2.1453,0.1226,1.7930,696.0,13472.0,70.0,10.0,17.896400,0.0000,63.187199,63.024502,0.0,375.108490,-0.0001,312.680786,317.798706,0.0,10052.0,27617.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181735,0.0305,1.1796,549.262512,1226.584961,32.0,80.820396,2472.643555,35.372898,37.157299,38.270599,0.0,-1.852400,58.112400,59.037800,180.854797,849.609375,1571.311890,2534.427979,311.317413,1112.160645,15.700700,15.5884,389.828003,41.083500,42.382801,12.0,11.6287,1.2903,0.7860,35.223400,35.574299,28.023800,1.0,0.0,1.0,0.0,793.0,0.0,11.6881,-4.1129,-1.7319,0.5802,1.5485,12944.0,-112.0,70.0,10.0,23.589399,-0.0002,-0.007200,0.614900,0.0,318.612488,-0.0019,319.082794,296.459106,0.0,10026.0,27593.0,0,0,0,0,1
181744,0.0305,1.2688,549.262512,1226.584961,32.0,81.276703,2550.893555,35.595699,37.548401,38.468899,0.0,-1.860000,58.256302,59.144600,179

time: 47.5 s


15

In [21]:
picklename = '/drive/My Drive/data/AAN_new/A1_A2/reset_Error_whole_x_test_15percent.pickle'
percent = 0.15
target = 2
max = 500 
target2 = 3
max2 = 1100

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
158867
에러 O
22933


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
7,0.0998,1.3746,541.599731,1207.342529,32.0,9.790400,157.107101,35.686901,12.435100,11.186200,100.0,96.375999,51.397598,51.988201,158.157303,799.163818,29.601999,154.128006,265.350311,1041.681763,12.195800,11.3235,403.731812,77.536301,78.996300,12.0,12.2238,1.3864,0.17720,35.467499,36.688202,28.020300,1.0,0.0,1.0,0.0,791.0,0.0,11.8201,-4.4535,-2.1344,-0.3707,1.3498,1552.0,13280.0,70.0,10.0,18.236401,0.0005,64.945000,61.812801,0.0,386.338898,0.0001,321.252899,325.484711,0.0,10053.0,27617.0,0,0,0,0,1
9,0.0998,1.3711,541.599731,1207.342529,32.0,9.839000,151.148804,35.678101,12.582700,12.223800,100.0,96.375999,51.766998,51.881401,156.631500,797.859314,29.029900,151.148804,265.159607,1033.390137,12.558300,11.3235,403.936310,77.508499,78.996300,12.0,12.2238,1.3864,0.17640,35.467499,36.688202,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.6508,-4.5111,-1.7845,0.7932,1.9398,1440.0,13360.0,70.0,10.0,18.211100,0.0005,64.699097,62.246799,0.0,385.742188,-0.0020,320.005096,324.580414,0.0,10053.0,27612.0,0,0,0,0,1
10,0.0998,1.3688,541.599731,1207.342529,32.0,9.822000,154.128006,35.672401,12.653100,12.254300,100.0,96.375999,51.756500,51.843300,156.440704,796.119690,28.648399,151.148804,264.015198,1034.346802,12.529000,11.3235,403.936310,77.488800,79.003899,12.0,12.2238,1.3864,0.18100,35.498001,36.688202,28.018600,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-3.8636,-1.5659,0.7002,1.5998,1368.0,13392.0,70.0,10.0,18.196600,0.0002,64.597801,62.427601,0.0,384.494385,-0.0001,317.852997,324.634613,0.0,10053.0,27617.0,0,0,0,0,1
15,0.0998,1.3621,541.599731,1207.342529,32.0,9.928200,151.148804,35.655602,12.887500,12.254300,100.0,96.391296,51.783699,51.835602,165.023804,811.775330,29.029900,155.120895,265.350311,1037.536011,12.539500,12.5061,404.345215,77.503799,79.003899,12.0,12.2314,1.3651,0.17640,35.467499,36.673000,28.022100,1.0,0.0,1.0,0.0,791.0,0.0,11.7151,-3.8580,-1.5607,0.6961,1.6043,1056.0,13472.0,70.0,10.0,18.037500,0.0002,63.975700,62.897900,0.0,379.195587,0.0013,315.086090,321.849701,0.0,10053.0,27614.0,0,0,0,0,1
17,0.0998,1.3564,541.599731,1207.342529,32.0,9.950200,151.148804,35.641499,13.007800,12.254300,100.0,96.375999,52.183102,51.835602,164.070099,810.470520,29.029900,154.128006,263.633698,1028.925415,12.925400,12.5214,404.396393,77.488098,79.003899,12.0,12.2238,1.3628,0.17490,35.467499,36.673000,28.019501,1.0,0.0,1.0,0.0,792.0,0.0,11.8188,-4.4556,-2.1329,-0.3630,1.3600,952.0,13472.0,70.0,10.0,17.968800,0.0002,63.722500,63.024502,0.0,379.032806,0.0008,315.285004,319.968903,0.0,10053.0,27617.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181758,0.0305,1.3574,549.262512,1226.584961,32.0,82.253601,2499.381592,35.817299,38.183899,38.972500,0.0,-1.852400,58.367901,59.030201,178.852051,847.869873,1606.597900,2556.331787,312.271088,1111.522827,16.404600,15.5884,389.725800,41.866600,42.375198,12.0,11.6287,1.2910,0.76385,35.208099,35.566700,28.023800,1.0,0.0,1.0,0.0,791.0,0.0,11.6954,-5.0605,-2.2912,0.0212,1.7023,9160.0,-208.0,70.0,10.0,29.756201,-0.0002,-0.007200,0.072400,0.0,357.928192,-0.0012,356.644196,342.556396,0.0,10026.0,27595.0,0,0,0,0,1
181763,0.0305,1.3859,549.262512,1226.584961,32.0,82.646301,2545.153320,35.888699,38.842701,38.980099,0.0,-1.852400,58.368050,59.03020

time: 1min 7s


30

In [22]:
picklename = '/drive/My Drive/data/AAN_new/A1_A2/reset_Error_whole_x_test_30percent.pickle'
percent = 0.3
target = 2
max = 500 
target2 = 3
max2 = 1100

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  

  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
135934
에러 O
45866


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
2,0.0998,1.3837,541.599731,1207.342529,32.0,9.704100,152.141693,35.709801,12.180500,11.186200,100.0,96.375999,51.424599,52.773998,159.110992,793.510376,29.029900,151.148804,269.927887,1042.000732,12.198100,11.3235,403.782898,77.600304,78.996300,12.0,12.2238,1.3872,0.1749,35.482800,36.688202,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8218,-4.4148,-2.1085,-0.3602,1.3557,2016.0,12976.0,70.0,10.0,18.268999,0.0007,65.357399,60.004299,0.0,391.167511,0.0008,325.900604,328.540894,0.0,10053.0,27616.0,0,0,0,0,1
3,0.0998,1.3854,541.599731,1207.342529,32.0,9.715200,154.128006,35.714001,12.247300,11.186200,100.0,96.375999,51.406399,52.773998,156.059296,801.773193,30.174200,151.148804,271.263092,1042.319580,12.170300,11.3235,403.782898,77.603798,79.003899,12.0,12.2238,1.3879,0.1772,35.482800,36.688202,28.024599,1.0,0.0,1.0,0.0,792.0,0.0,11.7147,-4.9002,-2.1456,0.1541,1.8032,1912.0,13056.0,70.0,10.0,18.243601,0.0005,65.328400,60.474499,0.0,390.606903,-0.0001,325.159088,328.215393,0.0,10053.0,27618.0,0,0,0,0,1
6,0.0998,1.3785,541.599731,1207.342529,32.0,9.779400,151.148804,35.696800,12.416400,11.186200,100.0,96.375999,51.425301,52.751202,157.966599,801.338196,29.601999,151.148804,269.546509,1041.043823,12.216300,11.3235,403.731812,77.561401,78.996300,12.0,12.2238,1.3872,0.1794,35.467499,36.688202,28.027201,1.0,0.0,1.0,0.0,793.0,0.0,11.8285,-3.7991,-1.7702,-0.0452,1.2192,1640.0,13248.0,70.0,10.0,18.229200,0.0007,65.039101,61.505402,0.0,384.928406,0.0014,320.384888,325.575104,0.0,10053.0,27614.0,0,0,0,0,1
7,0.0998,1.3746,541.599731,1207.342529,32.0,9.790400,157.107101,35.686901,12.435100,11.186200,100.0,96.375999,51.397598,51.988201,158.157303,799.163818,29.601999,154.128006,265.350311,1041.681763,12.195800,11.3235,403.731812,77.536301,78.996300,12.0,12.2238,1.3864,0.1772,35.467499,36.688202,28.020300,1.0,0.0,1.0,0.0,791.0,0.0,11.8201,-4.4535,-2.1344,-0.3707,1.3498,1552.0,13280.0,70.0,10.0,18.236401,0.0005,64.945000,61.812801,0.0,386.338898,0.0001,321.252899,325.484711,0.0,10053.0,27617.0,0,0,0,0,1
8,0.0998,1.3723,541.599731,1207.342529,32.0,9.820800,153.134705,35.681198,12.494100,11.506700,100.0,96.383698,51.688801,51.866100,157.394394,799.598572,29.029900,154.128006,264.587402,1034.665771,12.484600,11.3235,403.782898,77.522003,78.996300,12.0,12.2238,1.3864,0.1756,35.498001,36.673000,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-4.8966,-2.1376,0.1598,1.8016,1512.0,13328.0,70.0,10.0,18.236401,0.0005,64.822098,62.011700,0.0,386.465607,0.0014,320.999695,324.924103,0.0,10053.0,27617.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181783,0.0305,1.3083,549.262512,1226.584961,32.0,83.647598,2546.210693,35.694500,39.800400,38.980099,0.0,-1.852400,58.632599,59.030201,187.530502,866.569519,1589.431763,2545.757568,313.034088,1114.392944,17.010401,16.8930,390.339203,42.594799,42.375198,12.0,11.6287,1.2666,0.7494,35.192902,35.574299,28.022100,1.0,0.0,1.0,0.0,791.0,0.0,11.8255,-4.4244,-2.2173,-0.5527,1.1337,2592.0,4992.0,70.0,10.0,28.830299,0.0020,17.440701,25.354401,0.0,347.186005,-0.0006,327.600494,327.998413,0.0,10024.0,27588.0,0,0,0,0,1
181786,0.0305,1.3254,549.262512,1226.584961,32.0,83.828903,2539.563965,35.737202,39.957100,39.918499,0.0,-1.852400,58.452702,59.144600,184.0

time: 2min 12s


# **A1 A2 N**

5

In [23]:
picklename = '/drive/My Drive/data/AAN_new/A1_A2_N/reset_Error_whole_x_test_5percent.pickle'
percent = 0.05
target = 2
max = 500 
target2 = 3
max2 = 1100
target3 = 4
max3 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  avg3 = temp.iloc[:, target3].mean(axis=0) * avg_ratio
  
  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value3 = 0
    times = baesu
    while(new_value3 < max3 + avg3):
      new_value3 = x_test.iloc[j, target3] + (avg3*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target3] = new_value3  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
174156
에러 O
7644


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
8,0.0998,1.3723,541.599731,1207.342529,35.200001,9.820800,153.134705,35.681198,12.494100,11.506700,100.000000,96.383698,51.688801,51.866100,157.394394,799.598572,29.029900,154.128006,264.587402,1034.665771,12.4846,11.3235,403.782898,77.522003,78.996300,12.0,12.2238,1.3864,0.1756,35.498001,36.673000,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7152,-4.8966,-2.1376,0.1598,1.8016,1512.0,13328.0,70.0,10.0,18.236401,0.0005,64.822098,62.011700,0.0,386.465607,0.0014,320.999695,324.924103,0.0,10053.0,27617.0,0,0,0,0,1
15,0.0998,1.3621,541.599731,1207.342529,35.200001,9.928200,151.148804,35.655602,12.887500,12.254300,100.000000,96.391296,51.783699,51.835602,165.023804,811.775330,29.029900,155.120895,265.350311,1037.536011,12.5395,12.5061,404.345215,77.503799,79.003899,12.0,12.2314,1.3651,0.1764,35.467499,36.673000,28.022100,1.0,0.0,1.0,0.0,791.0,0.0,11.7151,-3.8580,-1.5607,0.6961,1.6043,1056.0,13472.0,70.0,10.0,18.037500,0.0002,63.975700,62.897900,0.0,379.195587,0.0013,315.086090,321.849701,0.0,10053.0,27614.0,0,0,0,0,1
57,0.0998,1.3764,541.599731,1207.342529,35.200001,10.441500,156.114197,35.691502,15.414800,14.276100,32.230598,94.331398,51.602001,53.010601,162.735001,817.863525,30.746500,154.128006,270.881592,1045.189697,11.5875,12.2543,407.514496,77.052498,78.286697,12.0,12.2238,1.3605,0.1772,35.482800,36.611900,28.026400,1.0,0.0,1.0,0.0,791.0,0.0,11.8460,-4.0689,-1.9619,-0.3208,1.2184,128.0,13824.0,70.0,10.0,17.357500,-0.0004,66.189201,63.838299,0.0,416.576294,-0.0008,351.074188,331.072815,0.0,10052.0,27614.0,0,0,0,0,1
63,0.0998,1.4254,541.599731,1207.342529,35.200001,10.728500,162.072495,35.813900,15.843500,14.466900,0.000000,83.299301,51.433201,53.010601,155.677795,793.510376,30.174200,159.093399,272.026001,1047.103271,11.3852,11.4151,407.361115,77.237801,78.286697,12.0,12.2238,1.3887,0.1756,35.482800,36.611900,28.024700,1.0,0.0,1.0,0.0,791.0,0.0,11.7854,-4.7061,-2.1870,-0.2334,1.5379,96.0,14432.0,70.0,10.0,17.303200,-0.0002,69.827904,66.406303,0.0,435.962799,-0.0003,367.910889,345.540314,0.0,10053.0,27619.0,0,0,0,0,1
99,0.0998,1.6321,541.599731,1207.342529,35.200001,13.509700,344.801605,36.330799,14.336200,16.580200,97.815102,39.567600,52.297401,52.072102,155.105499,791.336121,84.152199,391.477112,266.113190,1035.941284,11.7327,11.4380,407.463287,81.201698,81.605499,12.0,12.2238,1.3879,0.3854,35.482800,36.535599,28.026400,1.0,0.0,1.0,0.0,791.0,0.0,11.7018,-4.8688,-2.0941,0.2373,1.8419,16.0,16896.0,70.0,10.0,16.456900,0.0012,79.752602,78.721703,0.0,468.804291,0.0045,388.834686,390.172913,0.0,10053.0,27618.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181683,0.0305,1.2619,549.262512,1226.584961,35.200001,80.686096,2527.781250,35.578499,35.473499,37.141399,0.000000,-1.852400,57.625702,59.144600,179.138199,841.781616,1495.017944,2504.819824,313.224792,1115.349609,14.2150,14.1617,391.770386,45.954601,46.846001,12.0,11.6287,1.3506,0.8348,35.223400,35.604900,28.025499,1.0,0.0,1.0,0.0,791.0,0.0,11.6387,-4.9394,-2.1275,0.3736,1.8193,1800.0,9472.0,70.0,10.0,20.565701,0.0015,29.557301,51.522701,0.0,335.720490,-0.0017,305.736511,314.868988,0.0,10050.0,27601.0,0,0,0,0,1
181688,0.0305,1.2482,549.262512,1226.584961,35.200001,80.826599,2388.308105,35.554100,37.6106

time: 31.1 s


10

In [24]:
picklename = '/drive/My Drive/data/AAN_new/A1_A2_N/reset_Error_whole_x_test_10percent.pickle'
percent = 0.1
target = 2
max = 500 
target2 = 3
max2 = 1100
target3 = 4
max3 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  avg3 = temp.iloc[:, target3].mean(axis=0) * avg_ratio
  
  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value3 = 0
    times = baesu
    while(new_value3 < max3 + avg3):
      new_value3 = x_test.iloc[j, target3] + (avg3*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target3] = new_value3  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
166512
에러 O
15288


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
5,0.0998,1.3804,541.599731,1207.342529,35.200001,9.762300,151.148804,35.701401,12.329600,11.186200,100.0,96.375999,51.305099,52.773998,158.920303,794.380188,29.029900,153.134705,269.165009,1044.233032,12.117700,11.3159,403.731812,77.574402,79.003899,12.0,12.2238,1.3872,0.1764,35.482800,36.673000,28.025499,1.0,0.0,1.0,0.0,792.0,0.0,11.7141,-3.8737,-1.5734,0.6949,1.5980,1720.0,13184.0,70.0,10.0,18.261700,0.0005,65.191002,61.216000,0.0,388.798492,-0.0013,323.585785,326.913300,0.0,10053.0,27617.0,0,0,0,0,1
29,0.0998,1.3393,541.599731,1207.342529,35.200001,10.062200,151.148804,35.598801,13.645300,13.299600,100.0,96.375999,52.189201,52.949501,165.405197,812.645081,29.029900,158.100403,266.685486,1036.898193,12.727900,12.7426,405.674286,77.301598,79.011497,12.0,12.2238,1.3620,0.1741,35.467499,36.657700,28.021999,1.0,0.0,1.0,0.0,792.0,0.0,11.6479,-4.5810,-1.8256,0.7598,1.9617,528.0,13424.0,70.0,10.0,17.780701,0.0000,62.745899,62.897900,0.0,375.036194,0.0007,312.174500,316.008392,0.0,10053.0,27618.0,0,0,0,0,1
36,0.0998,1.3274,541.599731,1207.342529,35.200001,10.141400,151.148804,35.569000,14.150000,13.200400,100.0,96.375999,51.875900,53.010601,164.642303,815.689087,28.648399,151.148804,270.309387,1044.233032,12.277500,12.7426,406.185394,77.146103,78.996300,12.0,12.2238,1.3628,0.1764,35.467499,36.642399,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.8028,-3.7371,-1.6995,0.1108,1.2659,368.0,13360.0,70.0,10.0,17.690201,-0.0002,62.376999,62.590401,0.0,372.196899,-0.0006,310.022400,313.621185,0.0,10052.0,27617.0,0,0,0,0,1
43,0.0998,1.3166,541.599731,1207.342529,35.200001,10.160700,153.134705,35.541901,14.492900,13.208000,100.0,96.375999,51.810101,53.010601,164.070099,808.731201,29.411301,154.128006,271.072388,1043.914062,12.122300,12.7426,406.492188,76.955597,78.996300,12.0,12.2238,1.3612,0.1772,35.467499,36.627201,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.7439,-4.8497,-2.1730,-0.0176,1.7018,256.0,13312.0,70.0,10.0,17.559999,0.0000,62.116600,62.355301,0.0,371.708588,0.0009,310.167114,311.450989,0.0,10052.0,27616.0,0,0,0,0,1
78,0.0998,1.5459,541.599731,1207.342529,35.200001,11.745900,180.941498,36.115299,16.688801,15.580700,0.0,55.719002,51.984402,52.072102,157.585098,795.684814,37.231400,193.851501,264.778107,1033.708984,11.762000,11.4380,407.207794,78.600601,79.598999,12.0,12.2238,1.3895,0.2054,35.513302,36.581402,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8045,-4.5792,-2.1658,-0.3185,1.4392,40.0,15840.0,70.0,10.0,17.017500,0.0000,76.193604,73.061302,0.0,465.766113,0.0011,389.503815,373.010712,0.0,10053.0,27619.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181757,0.0305,1.3577,549.262512,1226.584961,35.200001,82.164398,2503.611328,35.818100,38.870701,38.865700,0.0,-1.852400,58.305302,59.030201,179.328903,842.651428,1607.742310,2509.200439,311.317413,1112.798340,16.317301,15.5884,389.623596,41.827202,42.375198,12.0,11.6287,1.2903,0.7730,35.208099,35.559101,28.024599,1.0,0.0,1.0,0.0,791.0,0.0,11.8006,-4.6964,-2.3263,-0.5317,1.2530,9640.0,-208.0,70.0,10.0,29.749001,0.0000,-0.007200,0.072400,0.0,354.926208,0.0001,355.197510,340.585205,0.0,10026.0,27601.0,0,0,0,0,1
181771,0.0305,1.3661,549.262512,1226.584961,35.200001,83.144096,2553.612793,35.839100,39.174999,38.980099,0.0,

time: 58.6 s


15

In [25]:
picklename = '/drive/My Drive/data/AAN_new/A1_A2_N/reset_Error_whole_x_test_15percent.pickle'
percent = 0.15
target = 2
max = 500 
target2 = 3
max2 = 1100
target3 = 4
max3 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  avg3 = temp.iloc[:, target3].mean(axis=0) * avg_ratio
  
  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value3 = 0
    times = baesu
    while(new_value3 < max3 + avg3):
      new_value3 = x_test.iloc[j, target3] + (avg3*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target3] = new_value3  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
158867
에러 O
22933


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,0.0998,1.3901,541.599731,1207.342529,35.200001,9.672400,149.162598,35.725800,12.025100,11.186200,100.0,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.1733,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,1
1,0.0998,1.3840,541.599731,1207.342529,35.200001,9.696700,150.155502,35.710499,12.156200,11.186200,100.0,96.375999,51.393002,52.773998,155.487000,799.163818,29.029900,149.162598,269.737213,1042.957397,12.136500,11.3235,403.680695,77.599503,79.003899,12.0,12.2238,1.3864,0.1779,35.498001,36.688202,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8256,-3.7916,-1.7599,-0.0161,1.2254,2120.0,12896.0,70.0,10.0,18.236401,0.0005,65.407997,59.570301,0.0,394.223785,-0.0027,325.882507,329.842987,0.0,10053.0,27617.0,0,0,0,0,1
16,0.0998,1.3589,541.599731,1207.342529,35.200001,9.938600,151.148804,35.647598,13.007000,12.254300,100.0,96.375999,51.917400,51.835602,162.925705,815.254211,29.601999,151.148804,261.917114,1034.665771,12.647800,12.5061,404.243011,77.500000,78.996300,12.0,12.2162,1.3643,0.1779,35.482800,36.657700,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.8299,-3.7922,-1.7679,-0.0526,1.2261,1008.0,13472.0,70.0,10.0,18.015800,0.0002,63.845501,62.970200,0.0,378.237091,-0.0019,316.550903,320.547607,0.0,10053.0,27613.0,0,0,0,0,1
21,0.0998,1.3495,541.599731,1207.342529,35.200001,9.995600,151.148804,35.624298,13.271600,12.254300,100.0,96.375999,52.111801,51.835602,166.740402,816.559021,28.457600,153.134705,265.159607,1034.027954,12.781800,12.7426,404.856415,77.441299,78.996300,12.0,12.2238,1.3635,0.1756,35.467499,36.673000,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.8301,-3.7905,-1.7644,-0.0515,1.2212,776.0,13472.0,70.0,10.0,17.896400,0.0000,63.375301,63.078701,0.0,376.211700,0.0001,315.393494,318.540192,0.0,10052.0,27618.0,0,0,0,0,1
24,0.0998,1.3413,541.599731,1207.342529,35.200001,10.020400,153.134705,35.603699,13.501000,12.254300,100.0,96.375999,52.123699,51.835602,165.214493,819.603088,29.220600,149.162598,264.587402,1035.622437,12.743400,12.7426,405.316498,77.370102,79.003899,12.0,12.2238,1.3628,0.1779,35.467499,36.657700,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.6509,-4.5300,-1.7967,0.7904,1.9448,664.0,13456.0,70.0,10.0,17.834900,0.0000,63.114899,63.024502,0.0,376.953094,-0.0008,311.993713,317.129608,0.0,10053.0,27614.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181735,0.0305,1.1796,549.262512,1226.584961,35.200001,80.820396,2472.643555,35.372898,37.157299,38.270599,0.0,-1.852400,58.112400,59.037800,180.854797,849.609375,1571.311890,2534.427979,311.317413,1112.160645,15.700700,15.5884,389.828003,41.083500,42.382801,12.0,11.6287,1.2903,0.7860,35.223400,35.574299,28.023800,1.0,0.0,1.0,0.0,793.0,0.0,11.6881,-4.1129,-1.7319,0.5802,1.5485,12944.0,-112.0,70.0,10.0,23.589399,-0.0002,-0.007200,0.614900,0.0,318.612488,-0.0019,319.082794,296.459106,0.0,10026.0,27593.0,0,0,0,0,1
181759,0.0305,1.3600,549.262512,1226.584961,35.200001,82.321404,2509.200439,35.823799,38.263802,38.980099,

time: 1min 25s


30

In [26]:
picklename = '/drive/My Drive/data/AAN_new/A1_A2_N/reset_Error_whole_x_test_30percent.pickle'
percent = 0.3
target = 2
max = 500 
target2 = 3
max2 = 1100
target3 = 4
max3 = 32

# A1: 2 / 500
# A2: 3 / 1100
# N:  4 / 32

avg_ratio = 0.1     # 평균의 반영비율
baesu = 1           # 평균*배수. 배수 설정.
sector_size = 200
sector_num = 909     

x_test = pd.read_pickle(os.getcwd() + '/drive/My Drive/data/AAN_new/reset_NoError_whole_x_test.pickle')
x_test.drop('time', axis=1, inplace=True)
 
for i in range(0, sector_num):
  sector_start = i * sector_size
  sector_end = sector_start + sector_size

  temp = x_test.iloc[sector_start:sector_end]   # 섹터추출
  normal_index = temp.loc[(temp.attack == 0)].index # 섹터에서 normal인 index 추출
  rand_index = random.sample(list(normal_index), int(len(normal_index)*percent))  # normal갯수*percent의 올림갯수만큼 random선택.   

  # 타겟에 더할 값 계산. avg = 평균*비율. avg는 평균이 아님.
  avg = temp.iloc[:, target].mean(axis=0) * avg_ratio
  avg2 = temp.iloc[:, target2].mean(axis=0) * avg_ratio
  avg3 = temp.iloc[:, target3].mean(axis=0) * avg_ratio
  
  # random index를 가지고 x_test에 에러 넣기
  for j in (rand_index):
    new_value = 0
    times = baesu
    while(new_value < max + avg):
      new_value = x_test.iloc[j, target] + (avg*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value2 = 0
    times = baesu
    while(new_value2 < max2 + avg2):
      new_value2 = x_test.iloc[j, target2] + (avg2*times)    # 에러의 세기 충분하도록 함
      times += 0.3

    new_value3 = 0
    times = baesu
    while(new_value3 < max3 + avg3):
      new_value3 = x_test.iloc[j, target3] + (avg3*times)    # 에러의 세기 충분하도록 함
      times += 0.3
    
    x_test.iloc[j, target] = new_value  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target2] = new_value2  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, target3] = new_value3  # 에러가 들어간 값으로 업데이트
    x_test.iloc[j, 63] = 1  # 에러플래그 1로

############################################### 저장 구간 ##################################################
# save as pickle here.
x_test.to_pickle(os.getcwd() + picklename)
print("pickle saved.")

# pickle test
a = pd.read_pickle(os.getcwd() + picklename)
print("")
print("로드된 데이터")
num0=0
num1=0
for i in a["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1

print('에러 X')
print(num0)
print('에러 O')
print(num1)


# Get names of indexes for which column Age has value 30
indexNames = a[a['error'] == 0 ].index
# Delete these row indexes from dataFrame
a.drop(indexNames , inplace=True)
a


pickle saved.

로드된 데이터
에러 X
135934
에러 O
45866


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,0.0998,1.3901,541.599731,1207.342529,35.200001,9.672400,149.162598,35.725800,12.025100,11.186200,100.0,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.1733,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,1
2,0.0998,1.3837,541.599731,1207.342529,35.200001,9.704100,152.141693,35.709801,12.180500,11.186200,100.0,96.375999,51.424599,52.773998,159.110992,793.510376,29.029900,151.148804,269.927887,1042.000732,12.198100,11.3235,403.782898,77.600304,78.996300,12.0,12.2238,1.3872,0.1749,35.482800,36.688202,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8218,-4.4148,-2.1085,-0.3602,1.3557,2016.0,12976.0,70.0,10.0,18.268999,0.0007,65.357399,60.004299,0.0,391.167511,0.0008,325.900604,328.540894,0.0,10053.0,27616.0,0,0,0,0,1
3,0.0998,1.3854,541.599731,1207.342529,35.200001,9.715200,154.128006,35.714001,12.247300,11.186200,100.0,96.375999,51.406399,52.773998,156.059296,801.773193,30.174200,151.148804,271.263092,1042.319580,12.170300,11.3235,403.782898,77.603798,79.003899,12.0,12.2238,1.3879,0.1772,35.482800,36.688202,28.024599,1.0,0.0,1.0,0.0,792.0,0.0,11.7147,-4.9002,-2.1456,0.1541,1.8032,1912.0,13056.0,70.0,10.0,18.243601,0.0005,65.328400,60.474499,0.0,390.606903,-0.0001,325.159088,328.215393,0.0,10053.0,27618.0,0,0,0,0,1
16,0.0998,1.3589,541.599731,1207.342529,35.200001,9.938600,151.148804,35.647598,13.007000,12.254300,100.0,96.375999,51.917400,51.835602,162.925705,815.254211,29.601999,151.148804,261.917114,1034.665771,12.647800,12.5061,404.243011,77.500000,78.996300,12.0,12.2162,1.3643,0.1779,35.482800,36.657700,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.8299,-3.7922,-1.7679,-0.0526,1.2261,1008.0,13472.0,70.0,10.0,18.015800,0.0002,63.845501,62.970200,0.0,378.237091,-0.0019,316.550903,320.547607,0.0,10053.0,27613.0,0,0,0,0,1
17,0.0998,1.3564,541.599731,1207.342529,35.200001,9.950200,151.148804,35.641499,13.007800,12.254300,100.0,96.375999,52.183102,51.835602,164.070099,810.470520,29.029900,154.128006,263.633698,1028.925415,12.925400,12.5214,404.396393,77.488098,79.003899,12.0,12.2238,1.3628,0.1749,35.467499,36.673000,28.019501,1.0,0.0,1.0,0.0,792.0,0.0,11.8188,-4.4556,-2.1329,-0.3630,1.3600,952.0,13472.0,70.0,10.0,17.968800,0.0002,63.722500,63.024502,0.0,379.032806,0.0008,315.285004,319.968903,0.0,10053.0,27617.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181786,0.0305,1.3254,549.262512,1226.584961,35.200001,83.828903,2539.563965,35.737202,39.957100,39.918499,0.0,-1.852400,58.452702,59.144600,184.097305,858.741821,1631.965576,2549.231934,310.936005,1110.885010,16.847500,16.8930,390.237000,42.756100,42.359901,12.0,11.6287,1.2651,0.7333,35.208099,35.559101,28.026400,1.0,0.0,1.0,0.0,792.0,0.0,11.6591,-4.3082,-1.7834,0.6677,1.6886,3984.0,3856.0,70.0,10.0,28.540899,0.0003,9.353300,20.001499,0.0,342.122406,-0.0011,333.025909,329.879211,0.0,10025.0,27593.0,0,0,0,0,1
181788,0.0305,1.3252,549.262512,1226.584961,35.200001,83.934700,2575.516846,35.736801,39.887299,40.223701,0.0,

time: 2min 52s
